In [1]:
from ib_insync import *
from datetime import datetime
import numpy as np
import pandas as pd
from trading_stratigies import intersection, sort_first_by_second, get_medium_vol_stocks, calculate_vol, write_file

In [2]:
# The function to get contract historical data
def get_contract_data(contract, look_back, resolution, side, end_time):
    bar = ib.reqHistoricalData(contract, endDateTime = end_time, durationStr=look_back,
        barSizeSetting= resolution, whatToShow=side, useRTH=True, formatDate=1)
# use util in ib-insync to convert bar data into pandas dataframe    
    df = util.df(bar)
    return df 

In [3]:
# The function to get stock historical data
def get_stock_data(ticker, look_back, resolution, side, end_time):
    contract = Contract(symbol = ticker, secType = 'STK', exchange = 'SMART',currency = 'USD')   
    if len(ib.qualifyContracts(contract)) == 0:
        return pd.DataFrame()
    return get_contract_data(contract, look_back, resolution, side, end_time)

In [4]:
# Connect to IB Server
ib=IB()
util.startLoop()
ib.connect('127.0.0.1',7497, clientId= 10)

<IB connected to 127.0.0.1:7497 clientId=10>

In [5]:
SectorNames = ("BasicMaterials, ConsumerCyclical, FinancialServices, RealEstate, ConsumerDefensive, Healthcare, \
Utilities, CommunicationServices, Energy, Industrials, Technology").split(', ')

The stock screening:

1 feed the stocks list selected by main.py (dollar_volume_min_price)

2 Find the intersection of lists (get stocks with continuous behavior, rule out temporarily hot stocks)

3 Use historical data to exculde top high-volatile stocks

In [6]:
# April 1 filtered_stocks (dollar_volume_min_price)
BasicMaterials = ['CLF' ,'X' ,'PX' ,'NUE' ,'RIO' ,'NEM' ,'FCX' ,'SHW' ,'ALB' ,'IST' ,'MLM' ,'ABX' ,'PPG' ,'VMC' ,'MOS' ,'APD' ,'ECL' ,'POT' ,'AA' ,'DOW' ,'GEVO' ,'STLD' ,'IFF' ,'RTP' ,'FVAC' ,'RKDA' ,'DOW' ,'CF' ,'BHP' ,'SLW' ,'GGB' ,'AMRS' ,'USCR' ,'LOAK' ,'CE' ,'EMN' ,'CTVA' ,'TCK' ,'AVTR' ,'CX' ,'BBL' ,'LAC' ,'KL' ,'FMC' ,'AXTA' ,'KGC' ,'FNV' ,'RS' ,'SUM' ,'AU' ,'SMG' ,'RPOW' ,'PLLL' ,'LYB' ,'CXP' ,'AEM' ,'PAASF' ,'PCU' ,'LTHM' ,'DRFNY' ,'OLN' ,'CC' ,'CRHCY' ,'HUN' ,'HL' ,'SQM' ,'MDU' ,'AG' ,'WFG' ,'KWR' ,'AUY' ,'RGLD' ,'CMC' ,'WLK' ,'FULL' ,'UFPI' ,'SBSW' ,'GPRE' ,'BTG' ,'UNVR' ,'UFS' ,'ASH' ,'CDE' ,'TROX' ,'FSM' ,'WDFC' ,'UFC' ,'AGI' ,'MTX' ,'TMST' ,'TSE' ,'KALU' ,'KRA' ,'SSRIF' ,'ESI' ,'POL' ,'HGMCY' ,'EXK' ,'CBT' ,'CMP' ,'IAG' ,'SID' ,'NGVT' ,'OTL' ,'EGO' ,'BCC' ,'SCL' ,'SWM' ,'EY' ,'BW' ,'FPGP' ,'PKX' ,'HCC' ,'GRA' ,'CENX' ,'ABH' ,'CSTM' ,'MWGP' ,'SCHN' ,'ZKIN' ,'BCP' ,'NG' ,'MVG' ,'RYAM' ,'DRJ' ,'SAND' ,'HBM' ,'ZEUS' ,'GSM' ,'NP' ,'SXC' ,'MEOHF' ,'SSLT' ,'UAMY' ,'ASIX' ,'URIX' ,'GCP' ,'PVG' ,'SVM' ,'HUGO' ,'SILV' ,'OR' ,'EQX' ,'BVN' ,'IPI' ,'SUZ' ,'KOP']
ConsumerCyclical = ['TSLA' ,'AMZN' ,'CHWY' ,'LULU' ,'GME' ,'BABA' ,'GM' ,'PCLN' ,'HD' ,'JD' ,'NKE' ,'PTON' ,'FTCH' ,'MCD' ,'F' ,'SBUX' ,'MELI' ,'LOW' ,'W' ,'CCL' ,'RH' ,'EBAY' ,'CPNG' ,'CMG' ,'DKNG' ,'KCAC' ,'EXPE' ,'MAR' ,'AZO' ,'NCLH' ,'PENN' ,'FD' ,'RCL' ,'TJX' ,'DLPH' ,'FIVE' ,'WSGC' ,'CVNA' ,'MGG' ,'HLT' ,'DHI' ,'ERI' ,'ORLY' ,'ULTA' ,'BLNK' ,'CTRP' ,'LVS' ,'BBY' ,'WYNN' ,'BURL' ,'LTD' ,'BLL' ,'UA' ,'DRI' ,'KMX' ,'TSCO' ,'DPZ' ,'SPAQ' ,'MGA' ,'GRWG' ,'YUM' ,'LEN' ,'PVH' ,'VTIQ' ,'RMG' ,'ROST' ,'YUMC' ,'ETSY' ,'WKHS' ,'KSS' ,'IP' ,'ASO' ,'HAS' ,'AAP' ,'GPS' ,'HDI' ,'VFC' ,'NVR' ,'COH' ,'SKX' ,'FBHS' ,'RL' ,'WHR' ,'LMTR' ,'UARM' ,'TRIP' ,'PHM' ,'DKS' ,'NOBE' ,'KORS' ,'Z' ,'GOEV' ,'OSTK' ,'QSR' ,'AIRB' ,'LEA' ,'GPC' ,'SFIX' ,'PII' ,'AEOS' ,'MTN' ,'WRK' ,'THO' ,'CROX' ,'MHK' ,'BYD' ,'CCK' ,'BWA' ,'VAC' ,'PLAY' ,'ARVL' ,'HTHT' ,'AMCR' ,'TOL' ,'WISH' ,'HBI' ,'RACE' ,'SIX' ,'ANF' ,'REAL' ,'BLMN' ,'EAT' ,'ATR' ,'YETI' ,'RII' ,'PLNT' ,'TXRH' ,'ROL' ,'VRM' ,'QRTEA' ,'WH' ,'RIDE' ,'FND' ,'TPX' ,'ALV' ,'BFAM' ,'PKG' ,'CBRL' ,'BC' ,'WING' ,'GNTX' ,'BBBY' ,'MUSA' ,'CONN' ,'SEE' ,'ABG' ,'POOL' ,'WOOF' ,'STAY' ,'XL' ,'HOME' ,'PZZA' ,'SERV' ,'MDC' ,'RVLV' ,'SIGYY' ,'ADNT' ,'ARMK' ,'HRB' ,'DECK' ,'LKQX' ,'WGO' ,'GT' ,'FLWS' ,'KBH' ,'VC' ,'CRI' ,'FCAU' ,'MTH' ,'FOXF' ,'VSTO' ,'TOYOY' ,'TKAT' ,'BZUN' ,'SHAK' ,'SRV' ,'FM' ,'ELY' ,'PLCE' ,'COLM' ,'IGT' ,'SHLL' ,'SEAS' ,'BERY' ,'CCS' ,'GPK' ,'DRVN' ,'LGIH' ,'CTB' ,'MED' ,'TRY' ,'LEG' ,'MAT' ,'H' ,'CUK' ,'FNKO' ,'SON' ,'CHDN' ,'GPI' ,'MPEL' ,'GAN' ,'BOOT' ,'TPH' ,'HGV' ,'OI' ,'TDUP' ,'CNNE' ,'WWW' ,'TUP' ,'CHGO' ,'URBN' ,'DAN' ,'DSW' ,'HZO' ,'FUV' ,'DDS' ,'FUN' ,'LESL' ,'NAKD' ,'TRWH' ,'CAKE' ,'DMYT' ,'GIII' ,'IHOP' ,'RUSHA' ,'GMBL' ,'MNRO' ,'PRTS' ,'LCII' ,'KTB' ,'CBD' ,'GOOS' ,'GES' ,'ALSN' ,'KAR' ,'POSH' ,'TTE' ,'BGFV' ,'UAG' ,'CWH' ,'PRTY' ,'SMP' ,'ARGNA' ,'SHOO' ,'EYE' ,'KIRK' ,'TTM' ,'DSEY' ,'HIBB' ,'ODP' ,'SBH' ,'TMHC' ,'RBIN' ,'CLAC' ,'EXPR' ,'WTW' ,'SLGN' ,'WPRT' ,'GCA' ,'FTDRV' ,'HMC' ,'BIOF' ,'LCA' ,'ACAM' ,'TCS' ,'RRGB' ,'SOLO' ,'BZH' ,'CHH' ,'DFXI' ,'RRR' ,'KNDI' ,'GPAC' ,'LZB' ,'SAH' ,'MHO' ,'DINE' ,'DOOO' ,'HOV' ,'AMWD' ,'REYN' ,'GP' ,'LL' ,'MRA' ,'MMYT' ,'VNE' ,'AXL' ,'OXM' ,'BKE' ,'MLHR' ,'GOLF' ,'SPWH' ,'SKY' ,'TEN' ,'HAVT' ,'RUTH' ,'FOSL' ,'CTRN' ,'CHUY' ,'GIL' ,'AYRO' ,'ZUMZ' ,'BG' ,'DLTH' ,'GBCOA' ,'PATK' ,'MBUU' ,'AIN' ,'LOVE' ,'LEVI' ,'VRNG' ,'PLYA' ,'CNGR' ,'ETH' ,'OCG' ,'BNED' ,'MCRI' ,'RNTA' ,'JWAIA' ,'RICK' ,'GCO' ,'CVCO' ,'FRSX' ,'FRG' ,'LEDMV' ,'CARS' ,'XPEL' ,'CPS' ,'SCVL' ,'LQDT' ,'MOVA' ,'DESP' ,'LAZY' ,'CRSV' ,'ONEW']
FinancialServices = ['V' ,'MARA' ,'PYPL' ,'MA' ,'GS' ,'BRKB' ,'NOB' ,'TRV' ,'CMB' ,'NB' ,'BRKA' ,'MWD' ,'BLK' ,'MHP' ,'SCH' ,'ACL' ,'TFC' ,'CCIV' ,'USB' ,'AOC' ,'PNC' ,'AXP' ,'CME' ,'PGR' ,'MET' ,'COF' ,'RKT' ,'PRU' ,'BK' ,'MMC' ,'HDB' ,'HIG' ,'SYF' ,'ALL' ,'SPC' ,'FITB' ,'SI' ,'ICE' ,'ITUB' ,'KEY' ,'STT' ,'CSR' ,'TROW' ,'WLTW' ,'FES' ,'AIG' ,'SIVB' ,'CFG' ,'DFS' ,'SBNY' ,'MKTX' ,'ALLY' ,'EQH' ,'RGBK' ,'BX' ,'UPST' ,'DNB' ,'LMND' ,'HBAN' ,'EBC' ,'AFL' ,'TCBI' ,'AMP' ,'PBCP' ,'FRC' ,'TD' ,'KKR' ,'AJG' ,'PBCT' ,'APO' ,'RCHI' ,'SLM' ,'WAL' ,'ROOT' ,'VIRT' ,'CMA' ,'EVR' ,'FNT' ,'IBN' ,'BBD' ,'NTRS' ,'ATH' ,'PFG' ,'RY' ,'ARES' ,'EWBC' ,'NDAQ' ,'BEN' ,'FFIN' ,'VOYA' ,'ING' ,'FTEN' ,'PSLV' ,'AVZ' ,'RJF' ,'LUK' ,'ADS' ,'FDS' ,'LPLA' ,'MXB' ,'RE' ,'WU' ,'CINF' ,'FRHC' ,'LNC' ,'ESPD' ,'DB' ,'TREE' ,'BNS' ,'BMO' ,'LPRO' ,'AIZ' ,'BCS' ,'ZION' ,'HBC' ,'CG' ,'CBOE' ,'CIT' ,'ESNT' ,'FAF' ,'MKL' ,'BKLY' ,'KNSL' ,'AFG' ,'RNR' ,'ARCC' ,'FCBP' ,'MTG' ,'BAP' ,'ISBC' ,'LC' ,'RGA' ,'BOWX' ,'BCM' ,'PFSI' ,'IBKR' ,'MFC' ,'UNM' ,'SLQT' ,'LTR' ,'HBANP' ,'AMG' ,'ORI' ,'XP' ,'TW' ,'EHTH' ,'Y' ,'PBR' ,'BPOP' ,'WTFC' ,'BHFWV' ,'CFR' ,'FCNCA' ,'PSTH' ,'TMK' ,'QCSB' ,'UBS' ,'PNFP' ,'SC' ,'SNV' ,'GDOT' ,'VLY' ,'AGO' ,'FBP' ,'CACC' ,'PHYS' ,'CBSH' ,'SLC' ,'TRUP' ,'AEL' ,'MORN' ,'SNPR' ,'SF' ,'ASB' ,'UMPQ' ,'PRSP' ,'CNO' ,'FHI' ,'FFC' ,'JHG' ,'TPGY' ,'WBST' ,'COWND' ,'BOKF' ,'LATN' ,'ACTC' ,'HLI' ,'SFNCA' ,'QELL' ,'AXS' ,'WD' ,'LFCO' ,'OZRK' ,'BKU' ,'MC' ,'APAM' ,'ERIE' ,'AFC' ,'OMF' ,'CMT' ,'FNC' ,'GHIV' ,'LAZ' ,'PRI' ,'CBU' ,'CHFC' ,'ABCB' ,'INDB' ,'UNIT' ,'LOB' ,'FOCS' ,'SAN' ,'FULT' ,'STD' ,'OLDB' ,'PLMR' ,'WSFS' ,'NAVIV' ,'CADE' ,'SEIC' ,'FBAN' ,'HBHC' ,'NMIH' ,'GBCI' ,'FLGS' ,'ARC' ,'CLII' ,'ACIC' ,'OMAM' ,'UBSI' ,'GSHD' ,'FRX' ,'AJAX' ,'KB' ,'EBC' ,'COLB' ,'ASPL' ,'WFSL' ,'HOL' ,'HLNE' ,'MCY' ,'UMBF' ,'IBTX' ,'PSEC' ,'SFTW' ,'CVB' ,'GOF' ,'IBOC' ,'ORCC' ,'GSVC' ,'MGI' ,'RTP' ,'BOH' ,'PRAA' ,'HOMB' ,'THBR' ,'UTF' ,'VRTS' ,'PFS' ,'PJT' ,'FSKR' ,'JRVR' ,'RA' ,'FFBC' ,'FSIC' ,'TBK' ,'FHB' ,'SIGI' ,'FMBI' ,'MAIN' ,'CNA' ,'CNS' ,'AC' ,'WPF' ,'NMR' ,'CATY' ,'BBV' ,'GNW' ,'SMFG' ,'FCF' ,'CFFN' ,'CEF' ,'MUFG' ,'AACQ' ,'TBBKD' ,'GHVI' ,'EIG' ,'FTOC' ,'CCX' ,'NARA' ,'BOFI' ,'VBTX' ,'RILY' ,'BPBC' ,'CMLF' ,'TYP' ,'PHC' ,'FRME' ,'VACQ' ,'SFBS' ,'WSBC' ,'PNTM.U' ,'ESGRD' ,'WTRE' ,'UCBI' ,'APSG' ,'GWB' ,'SPFR' ,'NMFC' ,'DSL' ,'FTCV' ,'PAWN' ,'MCMC' ,'FWWB' ,'RLI' ,'HYT' ,'UBSH' ,'PIPR' ,'ENVA' ,'GSAH' ,'ACEV' ,'AEG' ,'IAAC' ,'HMN' ,'CUBI' ,'EFC' ,'BXS' ,'OFG' ,'RMGB' ,'GOCO' ,'PDI' ,'UTG' ,'KYN' ,'PCI' ,'NEA' ,'THCB' ,'CASH' ,'NAD' ,'MAI' ,'FIBK' ,'EXG' ,'DFIN' ,'IPOF' ,'STIC' ,'PTY' ,'FFIC' ,'HTGC' ,'RBS' ,'CLM' ,'PXT' ,'NWSB' ,'JPS' ,'DCRB' ,'SCPE' ,'STEP' ,'BHL' ,'ASAQ' ,'TRMK' ,'PRK' ,'PRPB' ,'AMSF' ,'SHG' ,'JWS' ,'LFC' ,'SSPK' ,'SAFT' ,'LYG' ,'HTLF' ,'VGAC' ,'FSC' ,'WABC' ,'HPF' ,'SASR' ,'TWOA' ,'JIH' ,'FBNC' ,'GBDC' ,'BHFAN' ,'FSRV' ,'CPBI' ,'OCFC' ,'SBNYP' ,'PUK' ,'HEC' ,'EQOS' ,'BMA' ,'PFBC' ,'PAX' ,'BSBR' ,'BMEZ' ,'BRP' ,'BSTZ' ,'FAMCK' ,'DNP' ,'MOTV.U' ,'CHCO' ,'SBSI' ,'SBCFA' ,'CRHC' ,'FAX' ,'CFAC' ,'EDD' ,'NBTB' ,'EFSC' ,'HCII' ,'LKFN' ,'SPRQ' ,'BCX' ,'SRAC' ,'CIB' ,'STC' ,'INDI' ,'EGBN' ,'NBHC' ,'BRKL' ,'NTB' ,'INSP']
RealEstate = ['AMT' ,'TWRS' ,'SPG' ,'EQIXD' ,'PSA' ,'AMB' ,'RIGX' ,'UDR' ,'DLR' ,'ESS' ,'HCN' ,'ARE' ,'EXR' ,'OPEN' ,'SBAC' ,'VICI' ,'EQR' ,'BYA' ,'INVH' ,'O' ,'WY' ,'MAA' ,'RDFN' ,'NLY' ,'CDC' ,'VC' ,'PEAK' ,'AGNC' ,'EXPI' ,'ARCP' ,'BXP' ,'CBRE' ,'DRE' ,'FRT' ,'SUI' ,'MHC' ,'VNO' ,'REG' ,'MPW' ,'SLG' ,'OHI' ,'IIPR' ,'HR' ,'AMH' ,'CPT' ,'KIM' ,'COLD' ,'DOC' ,'BRX' ,'QTS' ,'MAC' ,'LAP' ,'DDR' ,'NRZ' ,'STOR' ,'NNN' ,'AIV' ,'YSI' ,'LSI' ,'AIRC' ,'HIW' ,'IMTN' ,'PK' ,'DEI' ,'ACC' ,'STWD' ,'REXR' ,'CONE' ,'CUZ' ,'GLPIV' ,'KRC' ,'STIR' ,'FR' ,'NSA' ,'MGP' ,'ADC' ,'HASI' ,'HPP' ,'CT' ,'HRP' ,'LAMR' ,'EGP' ,'HTA' ,'CLNY' ,'PEB' ,'COR' ,'LXP' ,'CIM' ,'CSALV' ,'SUNH' ,'SHO' ,'TRNO' ,'CBSO' ,'APLE' ,'NHI' ,'BDN' ,'PSM' ,'GET' ,'DEA' ,'COPT' ,'PCH' ,'RPAI' ,'PGRE' ,'ABR' ,'PDM' ,'WRE' ,'TWO' ,'SKT' ,'RLGY' ,'NAQ' ,'HHC' ,'EPR' ,'RYN' ,'FSRVF' ,'RLJ' ,'SRC' ,'WHC' ,'PMT' ,'JBGS' ,'EPRT' ,'DRH' ,'CTREV' ,'WRI' ,'ESRT' ,'MCT' ,'FCPT' ,'CXP' ,'EST' ,'LTC' ,'IVR' ,'BPY' ,'MFA' ,'AHT' ,'APT' ,'ARI' ,'XHR' ,'PZN' ,'AAT' ,'NTR' ,'PAX' ,'CWK' ,'RWTI' ,'SVC' ,'GNL' ,'UE' ,'IRT' ,'KRG' ,'ILPT' ,'SJP' ,'LADR' ,'ZFC' ,'ORC' ,'SAFE' ,'INN' ,'DX' ,'ACRE' ,'CLI' ,'RPT' ,'CLDT' ,'GPMT' ,'GTY' ,'NMRK' ,'BRMK' ,'ALEX' ,'UMH' ,'SNH' ,'BNL' ,'CMO' ,'LAND' ,'GMRE' ,'MNRTA' ,'FSV' ,'GOV' ,'NTST' ,'CHCT' ,'NXRT']
ConsumerDefensive = ['WMT' ,'COST' ,'PG' ,'WAFU' ,'PEP' ,'KR' ,'KO' ,'DH' ,'PM' ,'KMB' ,'KFT' ,'DG' ,'MO' ,'EL' ,'CL' ,'BYND' ,'DLTR' ,'GIS' ,'CBRNA' ,'CLX' ,'EDU' ,'DPS' ,'SAM' ,'HANS' ,'KHC' ,'TSN' ,'HSY' ,'CHD' ,'ADM' ,'MCCRK' ,'K' ,'CAG' ,'LW' ,'SYY' ,'FRPT' ,'TWOU' ,'BTI' ,'DAR' ,'CPB' ,'SJMA' ,'CHGG' ,'BFB' ,'SFM' ,'BUD' ,'UL' ,'BG' ,'EEIQ' ,'HRL' ,'CCEP' ,'BJ' ,'PFGC' ,'USFD' ,'ACCOB' ,'HELE' ,'COTY' ,'NWL' ,'CNS' ,'FMX' ,'HLF' ,'DEO' ,'CLEU' ,'BGS' ,'LOPE' ,'CELH' ,'OLLI' ,'FIZ' ,'CASY' ,'GO' ,'CPO' ,'ABEV' ,'POST' ,'UNFI' ,'HAIN' ,'THS' ,'FMCI' ,'EPC' ,'SAFM' ,'STRA' ,'NUS' ,'FLO' ,'HMHC' ,'JJSF' ,'NOMD' ,'CENTA' ,'ARCE' ,'LANC' ,'NOVS' ,'ACI' ,'LRN' ,'SPB' ,'PRMW' ,'OAC' ,'SMPL' ,'GRSH' ,'DV' ,'VITL' ,'VFF' ,'PDA' ,'NTCO' ,'CENT' ,'WPO' ,'NAPA' ,'PSMT' ,'USNA' ,'ELF' ,'COKE' ,'CALM' ,'BGL' ,'CHEF' ,'METX' ,'TPB' ,'PPC' ,'TR' ,'SPTN' ,'KOF' ,'JEAN' ,'UVV' ,'IMKTA' ,'CVGW' ,'LAUR' ,'STKLF' ,'AFYA' ,'UTZ' ,'WMK' ,'AQBTV' ,'CORE']
Healthcare = ['JNJ' ,'MRNA' ,'UNH' ,'ILMN' ,'BMY' ,'ABBV' ,'MRK' ,'TMO' ,'PFE' ,'LLY' ,'WAG' ,'AMGN' ,'MDT' ,'GILD' ,'TDOC' ,'ABT' ,'ATH' ,'ISRG' ,'NOX' ,'TLRY' ,'EXAS' ,'CVS' ,'DHR' ,'HUM' ,'REGN' ,'ALGN' ,'VEEV' ,'HCA' ,'EW' ,'IDPH' ,'HZNP' ,'BDX' ,'IDXX' ,'ALXN' ,'DXCM' ,'CRSP' ,'SYK' ,'NVS' ,'SNDL' ,'ZEN' ,'SGEN' ,'ZMH' ,'MCK' ,'VRTX' ,'PACB' ,'A' ,'BAX' ,'GLX' ,'WST' ,'TWST' ,'BSX' ,'GH' ,'VTRSV' ,'LH' ,'ZTS' ,'ICLRY' ,'HSGX' ,'IQV' ,'NVTA' ,'MRTX' ,'NTLA' ,'RESM' ,'HGEN' ,'KBIO' ,'CI' ,'PRAH' ,'BNGO' ,'PODD' ,'CNTE' ,'ELAN' ,'WAT' ,'CERN' ,'AAS' ,'HOLX' ,'BEAM' ,'MTD' ,'TXG' ,'BGNE' ,'CRL' ,'RGEN' ,'CAH' ,'ABCL' ,'CGC' ,'GDRX' ,'XRAY' ,'JAZZ' ,'FLGT' ,'SWAV' ,'FATE' ,'TECH' ,'UTHR' ,'NTRA' ,'AMED' ,'COO' ,'INCY' ,'AMWL' ,'TFX' ,'UHS' ,'SEEL' ,'NEO' ,'AVII' ,'DGX' ,'GNLN' ,'MASI' ,'BIOA' ,'EGG' ,'TGTX' ,'MOH' ,'QDEL' ,'EDIT' ,'STRL' ,'SDGR' ,'KOD' ,'BVF' ,'ASTM' ,'IOVA' ,'HTI' ,'ALNY' ,'INCR' ,'NBIX' ,'NVST' ,'TRL' ,'NVCR' ,'ABMD' ,'OMCL' ,'CDNA' ,'TNDM' ,'NARI' ,'CYH' ,'IRTC' ,'NNOX' ,'CTLT' ,'HQY' ,'NVO' ,'ACB' ,'CNAC' ,'VCYT' ,'ZLAB' ,'BMRN' ,'CNMD' ,'MRVI' ,'THC' ,'GEB' ,'RPRX' ,'HSIC' ,'PVTR' ,'PTIE' ,'AXNX' ,'ZOM' ,'ARGX' ,'BLI' ,'ARWR' ,'PTCT' ,'PEN' ,'XLRN' ,'BCRX' ,'RARE' ,'BLUE' ,'NSTG' ,'GRTS' ,'TEVIY' ,'DCTH' ,'PETS' ,'ACHC' ,'ENSG' ,'TAK' ,'NVRO' ,'SRNE' ,'INSP' ,'MMSI' ,'ARVN' ,'MGLN' ,'LIVN' ,'SNY' ,'KYMR' ,'ACAD' ,'CLVS' ,'CRIS' ,'ISIP' ,'CVAC' ,'NUVA' ,'NMTR' ,'TMDX' ,'OMI' ,'PHR' ,'ACRS' ,'PSNL' ,'SENS' ,'ARNA' ,'HB' ,'FGEN' ,'IGMS' ,'CHNG' ,'OCUL' ,'PPD' ,'EBS' ,'RAD' ,'STAA' ,'GKOS' ,'MEDP' ,'ENMD' ,'OSH' ,'AGIO' ,'HAE' ,'ALC' ,'EXEL' ,'CHE' ,'PRGO' ,'MRSN' ,'IPOC' ,'TPTX' ,'GBT' ,'HRC' ,'BPMC' ,'OCDX' ,'SHC' ,'BBIO' ,'CSTL' ,'AVXL' ,'FOLD' ,'INOV' ,'ITEK' ,'SEM' ,'CCXI' ,'NEOG' ,'SDC' ,'ADPT' ,'OSCR' ,'SEER' ,'CVETV' ,'PGNY' ,'VIR' ,'QGLY' ,'AVIR' ,'ITCI' ,'RCUS' ,'CRON' ,'CYPH' ,'HYH' ,'AHCO' ,'ATRC' ,'OM' ,'INSM' ,'TCRR' ,'GLPG' ,'AXSM' ,'AHS' ,'PDCO' ,'CERT' ,'ACCD' ,'EOLS' ,'FHC' ,'LUNG' ,'IART' ,'RUBY' ,'FMTX' ,'BHVN' ,'SGRY' ,'ALHC' ,'APLS' ,'EVH' ,'BDAL' ,'CNTL' ,'QTRX' ,'PRSC' ,'GB' ,'AHPA' ,'ZYME' ,'INMD' ,'GTXI' ,'EPIX' ,'ONEM' ,'INHL' ,'LHCG' ,'PAVM' ,'PHG' ,'CLN' ,'RLAY' ,'KURA' ,'SAGE' ,'PINC' ,'DCPH' ,'EYES' ,'ICPT' ,'RCM' ,'FREQ' ,'PRTA' ,'SRRK' ,'DNLI' ,'SWTX' ,'KITS' ,'GMED' ,'PDX' ,'AUPH' ,'SGFY' ,'BLFS' ,'SURF' ,'QGENF' ,'IMGN' ,'IRWD' ,'ARCT' ,'RETA' ,'ALKS' ,'GMAB' ,'ICUI' ,'MYGN' ,'PRQR' ,'AKBA' ,'SNDX' ,'OGI' ,'TCEL' ,'REPL' ,'ALLK' ,'MNKD' ,'DVAX' ,'MCRB' ,'RVMD' ,'PCRX' ,'CYTK' ,'BKD' ,'MASS' ,'HCAT' ,'KRTX' ,'LGND' ,'HCSG' ,'ANX' ,'CDXC' ,'ALT' ,'MGNX' ,'EPTO' ,'MDRX' ,'TCLN' ,'ANGO' ,'GTHX' ,'OPCH' ,'CCCC' ,'HSAC' ,'TNXP' ,'ALEC' ,'SGMO' ,'HEXO' ,'QURE' ,'ATEC' ,'HSKA' ,'MREO' ,'GRFS' ,'CNST' ,'RDYN' ,'TBIO' ,'CARA' ,'BTAI' ,'CERS' ,'AQST' ,'NABI' ,'MORF' ,'OPRX' ,'PBH' ,'PNTG' ,'CODX' ,'EAR' ,'RVNC' ,'AMHC' ,'TVTX' ,'PRVB' ,'ALXO' ,'HRTX' ,'CATS' ,'CORT' ,'ATRA' ,'RDY' ,'STRO' ,'CLVR' ,'ENDP' ,'ZGNX' ,'EPZM' ,'USPH' ,'KPTI' ,'ALLO' ,'AGEN' ,'SUPN' ,'PCVX' ,'DRNA' ,'TRHC' ,'TCMD' ,'MYOV' ,'KIN' ,'SILK' ,'ITRM' ,'CGEN' ,'RDUS' ,'APTO' ,'CCXX' ,'TRIL' ,'NRIX' ,'ARDX' ,'RGNX' ,'CHRS' ,'LNTH' ,'CBYL' ,'ATOS' ,'EWTX' ,'AFMD' ,'AMRX' ,'BNTC' ,'APR' ,'GBIO' ,'FLXN' ,'SIBN' ,'MRIC' ,'PLCC' ,'VNDA' ,'PETQ' ,'IKNA' ,'LMNX' ,'JYNT' ,'ADUS' ,'CRTX' ,'SYRS' ,'WVE' ,'KDMN' ,'SNN' ,'CDXS' ,'XNCR' ,'ESPR' ,'ANIK' ,'ORIC' ,'BDXB' ,'BOLT' ,'SANA' ,'KRG' ,'KALA' ,'RYTM' ,'XON' ,'AMTI' ,'SIMU' ,'CMRX' ,'DSGN' ,'NEOT' ,'TXMD' ,'GOSS' ,'ZNTL' ,'EBIO' ,'BDTX' ,'XERS' ,'CRMD' ,'NOVN' ,'PTGX' ,'YMAB' ,'BCAB' ,'PRLD' ,'AERI' ,'FBRX' ,'XXII' ,'LMAT' ,'ZIOP' ,'THLD' ,'STOK' ,'CLLS' ,'CRDF' ,'KRYS' ,'ALVR' ,'SIEN' ,'ATHA' ,'HCCO' ,'PMVP' ,'QSII' ,'OMER' ,'ITOS' ,'MTEC' ,'AVEO' ,'VREX' ,'BDSI' ,'KNSA' ,'ANVS' ,'NGM' ,'AAVL' ,'MBRX' ,'HIV' ,'XOMA' ,'VSTM' ,'INGN' ,'GLSI' ,'MDGL' ,'KIDS' ,'VOR' ,'RIGL' ,'IDYA' ,'SBTX' ,'CTMX' ,'VAPO' ,'TBPHV' ,'ATRI' ,'JAGX' ,'ATHX' ,'MBIO' ,'ATNX' ,'FMS' ,'ENTA' ,'CPRX' ,'KROS' ,'AKRO' ,'NKTX' ,'NEOT' ,'XENT' ,'FLDM' ,'STIM' ,'RNA' ,'SELB' ,'SRDX' ,'CGEM' ,'ADMA' ,'ONTX' ,'FDMT' ,'COLL' ,'VKTX' ,'INFU' ,'CFRX']
Utilities = ['FPL' ,'AEP' ,'NSP' ,'PPL' ,'ED' ,'PE' ,'D' ,'DUK' ,'PEG' ,'DTE' ,'AWK' ,'SO' ,'SRE' ,'EIX' ,'AES' ,'ETR' ,'NU' ,'CMS' ,'PCG' ,'NRG' ,'AEE' ,'FE' ,'HOU' ,'WPH' ,'WEC' ,'ABY' ,'ATO' ,'ORA' ,'VST' ,'WR' ,'PNW' ,'NI' ,'UGI' ,'WTRG' ,'IDA' ,'NGG' ,'SJI' ,'BEPC' ,'BEPCW' ,'NEP' ,'CIG' ,'OGE' ,'CWEN' ,'LG' ,'OEG' ,'OGS' ,'MPL' ,'WWP' ,'AGR' ,'POR' ,'PNM' ,'NWEC' ,'HE' ,'SWX' ,'NJR' ,'BKH' ,'ENI' ,'FTS' ,'AQN' ,'AEPPL' ,'BIP' ,'NFE' ,'ELP' ,'SOLN' ,'CWT' ,'SCW' ,'NWNG' ,'SPH' ,'CWEN.A' ,'SBS' ,'CPK' ,'BEP' ,'MSEX' ,'SJW' ,'MDSN' ,'AEPPZ' ,'BIPC' ,'OTTR' ,'MNTK' ,'DTP' ,'AZRE']
CommunicationServices = ['FB' ,'BIDU' ,'VIAC' ,'GOOCV' ,'GOOG' ,'NFLX' ,'DISCA' ,'SNAP' ,'ROKU' ,'DIS' ,'CMCSA' ,'SBC' ,'BEL' ,'ZM' ,'TWTR' ,'TWLO' ,'CHTR' ,'PINS' ,'Z' ,'ABNB' ,'BILI' ,'MTCH' ,'DASH' ,'BRIOF' ,'DISCK' ,'FUBO' ,'SPOT' ,'ERTS' ,'AMC' ,'RBLX' ,'FEAC' ,'FOXAV' ,'TTWO' ,'FVRR' ,'NTES' ,'IPG' ,'LBDKV' ,'LYV' ,'PCS' ,'ATUS' ,'ATVI' ,'OMC' ,'DISCB' ,'GNUS' ,'IAC' ,'CTL' ,'DISH' ,'ZG' ,'YNDX' ,'ZNGA' ,'CABO' ,'FOXBV' ,'NWSAV' ,'HOFV' ,'GRUB' ,'LBTYK' ,'CDRD' ,'GRPN' ,'NYT' ,'RUBI' ,'ATHM' ,'BCE' ,'CADC' ,'CNK' ,'VOD' ,'MSGE' ,'MSGEW' ,'GHQ' ,'CDLX' ,'LSXMK' ,'TU' ,'AIX' ,'YELP' ,'WMG' ,'COUP' ,'PLTK' ,'WWFE' ,'VIACA' ,'NXST' ,'GCI' ,'MSGS' ,'LMCKV' ,'CARG' ,'VG' ,'AMCX' ,'AMX' ,'RG' ,'NWSVV' ,'MBT' ,'LBTYA' ,'TTGT' ,'IDC' ,'SAQN' ,'LSXMA' ,'EROS' ,'COI' ,'TV' ,'SBGI' ,'TDS' ,'SJR' ,'DLPN' ,'LBDAV' ,'IZEA' ,'MSG' ,'SLGG' ,'SSTK' ,'SOGO' ,'SCPL' ,'SCHL' ,'JWA' ,'MAX' ,'DLX' ,'IHRT' ,'IMAXF' ,'GOGO' ,'MDP' ,'GCSB' ,'BOMN' ,'SCOR' ,'SKM' ,'VIV' ,'TIMB' ,'RUK' ,'LILKV' ,'NGMS' ,'QNST' ,'KTC' ,'SSP' ,'RADI' ,'LGF.B' ,'GCI' ,'SCR' ,'ORBC' ,'AVID' ,'LGF.A' ,'MOXC' ,'EVER' ,'SHET' ,'VIP' ,'WIFI' ,'BRAC' ,'CSN' ,'MAIL' ,'LIVX' ,'CHT' ,'LTPAV' ,'WOW' ,'TKC' ,'CNSL' ,'TEF' ,'VPRT' ,'MCS']
Energy = ['XON' ,'CHV' ,'P' ,'OXY' ,'PXD' ,'SLB' ,'KMI' ,'EOG' ,'PBR' ,'BP' ,'RDSA' ,'VLO' ,'MRO' ,'PSX' ,'MPC' ,'DVN' ,'FANG' ,'WMB' ,'EPD' ,'RDSB' ,'IPPIF' ,'HAL' ,'CCJ' ,'ETE' ,'AHC' ,'COG' ,'EQT' ,'TOT' ,'TPL' ,'REGI' ,'APA' ,'CED' ,'CHEX' ,'BHGE' ,'OKE' ,'SU' ,'AMTX' ,'CLNP' ,'MPLX' ,'AR' ,'CVE' ,'PAA' ,'HOC' ,'SWN' ,'DEN' ,'OVV' ,'PBRA' ,'WEG' ,'RIG' ,'TRP' ,'TRGP' ,'XEC' ,'MARY' ,'CLNE' ,'FTI' ,'PSXP' ,'CNX' ,'ETRN' ,'NOI' ,'MUR' ,'CLR' ,'PBF' ,'MTDR' ,'WGP' ,'PAGP' ,'HP' ,'LOM' ,'VVV' ,'UUUU' ,'DPM' ,'PETD' ,'DNN' ,'UEC' ,'PBA' ,'SRAQ' ,'CPG' ,'STO' ,'TS' ,'WLL' ,'STNG' ,'APY' ,'TPGE' ,'SHLX' ,'TRCH' ,'CHK' ,'AMGP' ,'ARCH' ,'PTEN' ,'DHT' ,'BCEI' ,'BTU' ,'LBRT' ,'RCON' ,'CDIS' ,'DSSI' ,'MPET' ,'GLNG' ,'SUSP' ,'SLCA' ,'WHD' ,'CRLBF' ,'LPI' ,'NRGY' ,'DK' ,'KOS' ,'UDL' ,'IMO' ,'OII' ,'OMP' ,'VET' ,'NBR' ,'NFG' ,'EXH' ,'INT' ,'SGY' ,'ENBL' ,'HEP' ,'EURN' ,'WTI' ,'EC' ,'OAS' ,'ENLC' ,'DNOW' ,'PARR' ,'CRK' ,'DRQ' ,'PUMP' ,'CVI' ,'CQP' ,'SNP' ,'MRC' ,'FRO' ,'GEL' ,'PTR' ,'SASOY' ,'NOG' ,'UGP' ,'NXE' ,'E' ,'ERF' ,'BPMP' ,'VNOM']
Industrials = ['BA' ,'PLUG' ,'CAT' ,'GE' ,'DE' ,'UNP' ,'AAL' ,'FDX' ,'ALD' ,'URI' ,'KSU' ,'CP' ,'ERC' ,'AYI' ,'LMT' ,'UTX' ,'UAUA' ,'DAL' ,'LUV' ,'MMM' ,'NOC' ,'CSX' ,'CARR' ,'GNRC' ,'CARRW' ,'AUD' ,'NSC' ,'GPN' ,'ETN' ,'ITW' ,'EMR' ,'CUM' ,'JCI' ,'ROP' ,'SPCE' ,'TDG' ,'UW' ,'RAIL' ,'SWK' ,'PH' ,'GD' ,'PAYX' ,'OTIS' ,'ROK' ,'OTISW' ,'HRS' ,'EFX' ,'ODFL' ,'GWW' ,'AME' ,'SPR' ,'UPS' ,'AOS' ,'CPRT' ,'PCAR' ,'MRKT' ,'CTAS' ,'EXPD' ,'FAST' ,'JBHT' ,'RSG' ,'VRSK' ,'JBLU' ,'MTZ' ,'BLDPF' ,'OC' ,'LII' ,'PWR' ,'XYL' ,'LPX' ,'WAB' ,'CHRW' ,'ZTO' ,'SAVE' ,'HII' ,'KNGT' ,'SNA' ,'TRU' ,'ACM' ,'IR' ,'CNI' ,'IEX' ,'AG' ,'PNR' ,'MIDD' ,'SITE' ,'AA' ,'BLDR' ,'VRT' ,'DOV' ,'CD' ,'WCNX' ,'SZI' ,'BE' ,'MAS' ,'GGG' ,'TXT' ,'FTIC' ,'RHI' ,'KBR' ,'WMS' ,'WGOV' ,'AVY' ,'TTC' ,'BLD' ,'ALK' ,'IIN' ,'PRLB' ,'HEI' ,'OTRKB' ,'LSTR' ,'WFII' ,'TASR' ,'SCST' ,'UPWK' ,'J' ,'TWP' ,'ALLE' ,'NDSN' ,'CFX' ,'AER' ,'BAH' ,'UHAL' ,'WSO' ,'GVA' ,'RYAAY' ,'HYFM' ,'AMRC' ,'HUBB' ,'TPIC' ,'RBC' ,'AEIS' ,'HTZ' ,'ACA' ,'TKR' ,'FLR' ,'AWI' ,'RBA' ,'ALGT' ,'MAN' ,'AAWW' ,'WATR' ,'MMS' ,'DCI' ,'ATKR' ,'NLSN' ,'ABB' ,'ROLL' ,'AQUA' ,'DY' ,'NAV' ,'AL' ,'UNF' ,'PSN' ,'XONE' ,'TAL' ,'WCC' ,'IVAC' ,'TEX' ,'HXL' ,'PRIM' ,'CSL' ,'APG' ,'GFL' ,'MSM' ,'KRNT' ,'EAF' ,'AAN' ,'ADT' ,'PRG' ,'BMI' ,'R' ,'AZEK' ,'HEIA' ,'AIMC' ,'NVT' ,'LECO' ,'BWC' ,'GY' ,'JBT' ,'ENS' ,'RXN' ,'TOC' ,'ABM' ,'EAGL' ,'TGI' ,'CW' ,'WBT' ,'BECN' ,'MIC' ,'GMT' ,'BW' ,'PAC' ,'SKYW' ,'CPA' ,'MRCY' ,'POLA' ,'HEES' ,'KEX' ,'MNES' ,'FAIL' ,'PZB' ,'ENR' ,'CWST' ,'ASGN' ,'ALT' ,'SWBI' ,'BWC' ,'MYRG' ,'BWEN' ,'GBX' ,'WERN' ,'EMCG' ,'WTHG' ,'RCII' ,'AVAV' ,'REZI' ,'CNHI' ,'BLDG' ,'ARNC' ,'ALEX' ,'TRN' ,'VALM' ,'HAYW' ,'SRCL' ,'KIQ' ,'PFMT' ,'SSD' ,'CLHB' ,'FLS' ,'PBI' ,'ERJ' ,'NAT' ,'KMT' ,'FRTA' ,'DAC' ,'ASF' ,'IAA' ,'HI' ,'ROCK' ,'TMI' ,'AZUL' ,'DOOR' ,'MIII' ,'KFY' ,'IBP' ,'GMS' ,'ABFS' ,'ARLO' ,'KODK' ,'CR' ,'MEG' ,'ASR' ,'VTSI' ,'TNET' ,'GSHT' ,'HONI' ,'MWA' ,'AMSC' ,'AAON' ,'GOL' ,'CYRX' ,'FIX' ,'OMAB' ,'LAND' ,'TFT' ,'SBLK' ,'CRS' ,'JELD' ,'TFII' ,'AQMS' ,'TGH' ,'PCR' ,'SCS' ,'SINO' ,'MLI' ,'ICFI' ,'DHC' ,'CSWIV' ,'WTS' ,'BRCOA' ,'HA' ,'RGR' ,'AZZ' ,'POWW' ,'SNDR' ,'ENRJ' ,'FELE' ,'INSW' ,'MTW' ,'SPW' ,'CATM' ,'ROAD' ,'ASTE' ,'HUBG' ,'FLOW' ,'B' ,'CMCO' ,'HURN' ,'WNC' ,'APZ' ,'AGX' ,'VRTV' ,'FSS' ,'MESA' ,'CBIZ' ,'EPAC' ,'NVEE' ,'OPTT' ,'RAVN' ,'MOG.A' ,'LNN' ,'DPW' ,'EOSE' ,'CGT' ,'AIR' ,'FTAI' ,'WIRE' ,'OAKC' ,'CAP' ,'ABXA' ,'CFXA' ,'ACP' ,'HSII' ,'TANT' ,'KNL' ,'GFF' ,'SFL' ,'SXI' ,'VVI' ,'CBAK' ,'SPAB' ,'DCO' ,'ALG' ,'TRNS' ,'CRAI' ,'CIR' ,'HWCC' ,'HTLD' ,'APOG' ,'MGRC' ,'NMM' ,'MRTN' ,'GTES' ,'NX' ,'SHYF' ,'NXTD' ,'PGTI' ,'TWI' ,'HSC' ,'REVG' ,'KELYA' ,'ECHO' ,'DGLY' ,'KAMNA' ,'SGBX' ,'SNCY' ,'WLDN' ,'IDS' ,'ACTG' ,'SNHY' ,'VVNT' ,'GLDD' ,'IEE' ,'NPO' ,'USX' ,'USAK' ,'ROMC' ,'HYRE' ,'LBOR']
Technology = ['AAPL' ,'MSFT' ,'NVDA' ,'AMD' ,'AMAT' ,'SQ' ,'INTC' ,'MU' ,'SHOP' ,'CRM' ,'ADBE' ,'LRCX' ,'PLTR' ,'TSM' ,'APPY' ,'CSCO' ,'MVIS' ,'QCOM' ,'TXN' ,'ASMLF' ,'SNOW' ,'UBER' ,'CRWD' ,'AVGO' ,'NOW' ,'FISV' ,'ACN' ,'NXPI' ,'KLAC' ,'IBM' ,'INTU' ,'ORCL' ,'ADI' ,'TTD' ,'ENPH' ,'RNG' ,'ADSK' ,'CEY' ,'DDOG' ,'AHA' ,'DOCU' ,'DOX' ,'RUN' ,'COUP' ,'OKTA' ,'LYFT' ,'SNPS' ,'U' ,'PANW' ,'MCHP' ,'MNDL' ,'WDAY' ,'MSTR' ,'TEAM' ,'CDN' ,'WDC' ,'ON' ,'ZS' ,'SPLK' ,'MDB' ,'TER' ,'PLAN' ,'HUBS' ,'RIMM' ,'CTSH' ,'UBNT' ,'AVLR' ,'AI' ,'VUZI' ,'STNE' ,'MXIM' ,'HWP' ,'TEL' ,'WIX' ,'XLNX' ,'FSLR' ,'STX' ,'QRVO' ,'FTV' ,'ANSS' ,'GLW' ,'NET' ,'SEDG' ,'FLT' ,'EPAM' ,'FSLY' ,'SPWR' ,'TDY' ,'FTNT' ,'INFY' ,'CTXS' ,'APH' ,'CHKPF' ,'PAYC' ,'NOVA' ,'EBON' ,'HPE' ,'CREE' ,'ZBRA' ,'MRVL' ,'AKAM' ,'ZEN' ,'WORK' ,'DELL' ,'ENTG' ,'AFRM' ,'FIVN' ,'MPWR' ,'KEYS' ,'TRMB' ,'DBX' ,'FFIV' ,'GART' ,'BILL' ,'ARW' ,'VSNR' ,'CDW' ,'PMTC' ,'NTAP' ,'AYX' ,'SLTD' ,'FIC' ,'VMW' ,'COHR' ,'VRSN' ,'FAF' ,'BIGC' ,'MOT' ,'IIVI' ,'SAP' ,'ESTC' ,'GDS' ,'DXC' ,'TYL' ,'SSNC' ,'QTWO' ,'GWRE' ,'PSTG' ,'GDDY' ,'IRBT' ,'LITEV' ,'VRNT' ,'BMBL' ,'APPN' ,'CPBY' ,'PAGS' ,'DT' ,'BL' ,'CSIQ' ,'PANL' ,'BRKS' ,'WXS' ,'CDAY' ,'GRMN' ,'PCTY' ,'TDSC' ,'LPSN' ,'JKHY' ,'SAI' ,'NOK' ,'BR' ,'BAND' ,'CGNX' ,'CIEN' ,'GMHI' ,'LAZR' ,'OLO' ,'AZPN' ,'KOPN' ,'PD' ,'SABR' ,'ANET' ,'SYMC' ,'BOX' ,'MKSI' ,'LSCC' ,'FLEXF' ,'UNPH' ,'CLDR' ,'MDLA' ,'VRNS' ,'FEYE' ,'SHLS' ,'ARRY' ,'BKI' ,'SWCH' ,'TENB' ,'CRNCV' ,'KLIC' ,'ST' ,'SYNA' ,'G' ,'STM' ,'SMAR' ,'JFAX' ,'NEWR' ,'ERICY' ,'RXT' ,'ITRI' ,'CLSK' ,'MDA' ,'UCTT' ,'MANH' ,'NCNO' ,'YOD' ,'SAIL' ,'LOGIY' ,'VZIO' ,'AVGOP' ,'DNB' ,'SEAC' ,'JNPR' ,'SONO' ,'GPRO' ,'FOUR' ,'PFPT' ,'EEFT' ,'CCC' ,'TRNE' ,'STMP' ,'ZI' ,'VNT' ,'CRUS' ,'VNTW' ,'NICEY' ,'LFUS' ,'DOMO' ,'SAIC' ,'BSY' ,'WKEY' ,'BEEM' ,'VLDR' ,'JBIL' ,'EVBG' ,'CNXC' ,'AMKR' ,'PEGA' ,'CACI' ,'KOSS' ,'ASAN' ,'DLB' ,'RPD' ,'CDKVV' ,'AMBA' ,'UMC' ,'NCR' ,'XRX' ,'ENV' ,'SNX' ,'IPGP' ,'QLYS' ,'SSYS' ,'EGHT' ,'CCMP' ,'CRCT' ,'NSIT' ,'CYBR' ,'FROG' ,'CALX' ,'CRSR' ,'LSPD' ,'SLAB' ,'GLOB' ,'NTNX' ,'TSAI' ,'POWI' ,'GTNR' ,'MTSI' ,'DIO' ,'AVYA' ,'TTEC' ,'PING' ,'PTC' ,'TLS' ,'SPT' ,'AVT' ,'NPTN' ,'SITM' ,'ICHR' ,'SUMO' ,'COMM' ,'NVMM' ,'TDC' ,'ONTO' ,'ACMR' ,'ALRM' ,'CVLT' ,'EXLS' ,'XM' ,'FORM' ,'EB' ,'DCT' ,'VSAT' ,'NVTL' ,'DNJR' ,'SMTC' ,'ACXM' ,'GIB' ,'APPF' ,'MANT' ,'CSOD' ,'EVTC' ,'ZUO' ,'ATMR' ,'CMBM' ,'PUBM' ,'FN' ,'MXL' ,'UIS' ,'GSCN' ,'SPSC' ,'DOCN' ,'WK' ,'CGNT' ,'NTCT' ,'VSCI' ,'NATI' ,'PTAC' ,'MCFE' ,'DSP' ,'ACLS' ,'PLXS' ,'MX' ,'PCVA' ,'OTEXF' ,'MIME' ,'BLKB' ,'ASX' ,'GILTF' ,'PRFT' ,'SANM' ,'EPAY' ,'ALTR' ,'VSH' ,'INTZ' ,'SMCI' ,'TSEMF' ,'SPU' ,'VCRA' ,'YEXT' ,'ALGM' ,'EMA' ,'PIOS' ,'COHU' ,'INFN' ,'MODN' ,'PRGS' ,'SVMK' ,'TTMI' ,'TSRA' ,'KN' ,'DBD' ,'ROG' ,'MAXNV' ,'SWI' ,'VERI' ,'SATS' ,'XGTI' ,'MOGO' ,'VVPR' ,'CSGS' ,'DLPH' ,'NTGR' ,'PRO' ,'EXTR' ,'TBRG' ,'CNDT' ,'ACVA' ,'PLAB' ,'WIT' ,'UPLD' ,'ESE' ,'CPSH' ,'RMBS' ,'OSIS' ,'SGH' ,'PAMT' ,'BHE' ,'HSTX' ,'DMRC' ,'LASR' ,'VICR' ,'MFGP' ,'ATC' ,'VERX' ,'CAZA' ,'MICT' ,'LPL' ,'SMSI' ,'LCA' ,'JAMF' ,'MLCH' ,'METHA' ,'ONTF' ,'EVOP' ,'VECO' ,'MITK' ,'BCOV' ,'ECOM' ,'IMMR' ,'AOSL' ,'PAYA' ,'SMMC' ,'GBOX' ,'SYKE' ,'VDSI' ,'AKTS' ,'IIIV' ,'CMTL' ,'SSNT' ,'WATT' ,'JSYN' ,'GPIA' ,'AXTI' ,'ZDGE' ,'CTS' ,'RDWR' ,'PI' ,'NVMI' ,'MLAB']
April_1 = [BasicMaterials, ConsumerCyclical, FinancialServices, RealEstate, ConsumerDefensive, Healthcare,\
           Utilities, CommunicationServices, Energy, Industrials, Technology]

In [7]:
# May 4 filtered_stocks (dollar_volume_min_price)
BasicMaterials = ['FCX' ,'X' ,'RIO' ,'NUE' ,'NEM' ,'DOW' ,'ABX' ,'CLF' ,'DOW' ,'SHW' ,'APD' ,'MOS' ,'PX' ,'LYB' ,'RTP' ,'AVTR' ,'AA' ,'GGB' ,'MLM' ,'ECL' ,'STLD' ,'AEM' ,'IST' ,'PPG' ,'IFF' ,'CTVA' ,'ALB' ,'VMC' ,'SLW' ,'OLN' ,'CF' ,'KGC' ,'FMC' ,'AU' ,'GRA' ,'PAASF' ,'BHP' ,'TCK' ,'AXTA' ,'HL' ,'PCU' ,'EMN' ,'AG' ,'POT' ,'CE' ,'FNV' ,'GEVO' ,'CDE' ,'WLK' ,'SMG' ,'FSM' ,'FVAC' ,'HUN' ,'SID' ,'BBL' ,'LOAK' ,'RPOW' ,'CXP' ,'KL' ,'ESI' ,'AMRS' ,'RS' ,'AUY' ,'CX' ,'BCP' ,'CC' ,'CMC' ,'RGLD' ,'PLLL' ,'DRFNY' ,'SCHN' ,'WFG' ,'SBSW' ,'SQM' ,'TROX' ,'NGVT' ,'GPRE' ,'LAC' ,'ASH' ,'ABH' ,'CENX' ,'SSRIF' ,'UFPI' ,'BTG' ,'SVM' ,'EQX' ,'TSE' ,'AGI' ,'MVG' ,'FULL' ,'UFS' ,'LTHM' ,'IAG' ,'SUM' ,'POL' ,'ZEUS' ,'HGMCY' ,'EGO' ,'MDU' ,'EXK' ,'CBT' ,'BCC' ,'HUGO' ,'WDFC' ,'CMP' ,'PQG' ,'SAND' ,'CRHCY' ,'CSTM' ,'EY' ,'UFC' ,'HCC' ,'ZY' ,'SCL' ,'SUZ' ,'BVN' ,'ASIX' ,'BW' ,'MEOHF' ,'PVG' ,'BAK' ,'USGL' ,'KRA' ,'UNVR' ,'FPGP' ,'NG' ,'SILV' ,'DRJ' ,'KALU' ,'KWR' ,'SSLT' ,'PKX' ,'TMST' ,'USCR' ,'OEC' ,'HBM' ,'SXC' ,'MERCS' ,'OR' ,'FRD' ,'SA' ,'MWGP' ,'MTX' ,'KOP' ,'GSM' ,'SWM' ,'TKOCF' ,'GATO' ,'OTL' ,'ASM' ,'GCP' ,'URIX' ,'NAK' ,'IPI']
ConsumerCyclical = ['AMZN' ,'TSLA' ,'BABA' ,'HD' ,'DKNG' ,'GME' ,'EBAY' ,'ETSY' ,'F' ,'GM' ,'PTON' ,'LOW' ,'CCL' ,'MCD' ,'SBUX' ,'NCLH' ,'NKE' ,'MELI' ,'PCLN' ,'GPS' ,'W' ,'MHK' ,'JD' ,'PENN' ,'CVNA' ,'CHWY' ,'CMG' ,'DPZ' ,'EXPE' ,'TJX' ,'COH' ,'FD' ,'RH' ,'ORLY' ,'DHI' ,'LULU' ,'KSS' ,'LTD' ,'RCL' ,'MAR' ,'DLPH' ,'ULTA' ,'Z' ,'MGG' ,'LVS' ,'OSTK' ,'LEN' ,'SPAQ' ,'AZO' ,'WYNN' ,'KCAC' ,'FTCH' ,'VFC' ,'HLT' ,'CTRP' ,'ERI' ,'UARM' ,'PHM' ,'DKS' ,'AAP' ,'WHR' ,'FIVE' ,'TSCO' ,'BURL' ,'KMX' ,'BBY' ,'ARMK' ,'TRIP' ,'IP' ,'RL' ,'RII' ,'CROX' ,'ROST' ,'DRI' ,'YUMC' ,'POOL' ,'YUM' ,'WSGC' ,'TTE' ,'KORS' ,'NOBE' ,'AEOS' ,'SEE' ,'CPNG' ,'URBN' ,'CRI' ,'BWA' ,'WKHS' ,'FBHS' ,'MGA' ,'BC' ,'BLL' ,'RACE' ,'GPC' ,'CCK' ,'TXRH' ,'QSR' ,'TPX' ,'PVH' ,'NAKD' ,'VTIQ' ,'WING' ,'AMCR' ,'BFAM' ,'DECK' ,'SKX' ,'WRK' ,'MTN' ,'HAS' ,'KAR' ,'CWH' ,'YETI' ,'LMTR' ,'PLNT' ,'LEVI' ,'EAT' ,'HBI' ,'SRV' ,'BBBY' ,'RVLV' ,'TOL' ,'SIX' ,'FND' ,'BERY' ,'LKQX' ,'WISH' ,'GT' ,'BLNK' ,'PII' ,'HGV' ,'HDI' ,'CCS' ,'COLM' ,'NVR' ,'ANF' ,'SEAS' ,'MPEL' ,'FLWS' ,'ARVL' ,'ELY' ,'KTB' ,'ROL' ,'SHAK' ,'WTW' ,'LEG' ,'BLMN' ,'SIGYY' ,'VAC' ,'DDS' ,'MAT' ,'MTH' ,'CHDN' ,'GRWG' ,'LESL' ,'LGIH' ,'UA' ,'HRB' ,'BYD' ,'LEA' ,'CAKE' ,'MHO' ,'PKG' ,'KBH' ,'FOXF' ,'MDC' ,'DAN' ,'TKAT' ,'SFIX' ,'RRR' ,'GNTX' ,'AIRB' ,'WOOF' ,'CTB' ,'H' ,'RIDE' ,'HZO' ,'THO' ,'TMHC' ,'PLAY' ,'WGO' ,'PLCE' ,'REAL' ,'SBH' ,'ATR' ,'VRM' ,'GPK' ,'PZZA' ,'TOYOY' ,'MUSA' ,'VC' ,'ASO' ,'CBRL' ,'XL' ,'QRTEA' ,'DSEY' ,'SHLL' ,'UAG' ,'WH' ,'MED' ,'HIBB' ,'ALSN' ,'VSTO' ,'HMC' ,'HTHT' ,'GOOS' ,'GCA' ,'TPH' ,'FNKO' ,'HOME' ,'GPAC' ,'BIOF' ,'SOLO' ,'TCS' ,'ALV' ,'ADNT' ,'GIL' ,'STAY' ,'RMG' ,'FCAU' ,'IGT' ,'KIRK' ,'FM' ,'DSW' ,'SERV' ,'SHOO' ,'TRY' ,'MCFT' ,'DMYT' ,'TUP' ,'PRTY' ,'IHOP' ,'EYE' ,'BG' ,'MLHR' ,'BOOT' ,'CHH' ,'FUN' ,'SON' ,'LOVE' ,'ABG' ,'BZH' ,'REYN' ,'ETH' ,'LZB' ,'WWW' ,'CUK' ,'GAN' ,'TRWH' ,'DOOO' ,'GES' ,'BGFV' ,'GPI' ,'HAVT' ,'CTRN' ,'BKE' ,'SLGN' ,'WPRT' ,'DFXI' ,'RBIN' ,'LCII' ,'GIII' ,'RNTA' ,'ACAM' ,'CNNE' ,'SKY' ,'PRTS' ,'SAH' ,'GMBL' ,'VNE' ,'OI' ,'ODP' ,'BZUN' ,'CONN' ,'TTM' ,'CVCO' ,'PATK' ,'ARGNA' ,'GBCOA' ,'CHGO' ,'POSH' ,'FTDRV' ,'AXL' ,'CBD' ,'MRA' ,'HOV' ,'LCA' ,'GOEV' ,'LL' ,'MBUU' ,'FRG' ,'FRSX' ,'FUV' ,'EXPR' ,'AIN' ,'LENB' ,'DRVN' ,'KNDI' ,'MCRI' ,'MMYT' ,'RUSHA' ,'MOVA' ,'DINE' ,'AMWD' ,'CHUY' ,'NCT' ,'ONEW' ,'GDEN' ,'BSET' ,'OXM' ,'GOLF' ,'RRGB' ,'ZUMZ' ,'CARS' ,'SPWH' ,'RUTH' ,'MNRO' ,'JWAIA' ,'FOSL' ,'TEN' ,'RICK' ,'INSU' ,'CSPR' ,'WNW' ,'CNGR' ,'XPEL' ,'LOCO' ,'GCO' ,'JOAN' ,'RCKY' ,'MYE' ,'AYRO']
FinancialServices = ['PYPL' ,'V' ,'MA' ,'BRKA' ,'BRKB' ,'TRV' ,'CMB' ,'NB' ,'GS' ,'NOB' ,'MARA' ,'MWD' ,'MHP' ,'COF' ,'AXP' ,'AOC' ,'SCH' ,'BLK' ,'ACL' ,'USB' ,'ICE' ,'TFC' ,'MMC' ,'CCIV' ,'ITUB' ,'SYF' ,'DFS' ,'PNC' ,'PGR' ,'DNB' ,'MET' ,'UPST' ,'BK' ,'BX' ,'ALL' ,'CME' ,'PRU' ,'SI' ,'CFG' ,'SIVB' ,'ALLY' ,'IBN' ,'TROW' ,'SPC' ,'FITB' ,'WLTW' ,'OMF' ,'AFL' ,'PSLV' ,'HBAN' ,'BBD' ,'HIG' ,'AMP' ,'RGBK' ,'KEY' ,'APO' ,'HDB' ,'RKT' ,'LMND' ,'EQH' ,'WU' ,'AIG' ,'TD' ,'AJG' ,'KKR' ,'RE' ,'SBNY' ,'BAP' ,'PBCT' ,'LPLA' ,'CMA' ,'NDAQ' ,'STT' ,'ADS' ,'CINF' ,'FES' ,'SLM' ,'BEN' ,'PFG' ,'QCSB' ,'RJF' ,'MXB' ,'RY' ,'NTRS' ,'AVZ' ,'MKTX' ,'RNR' ,'MTG' ,'CACC' ,'ZION' ,'ATH' ,'LNC' ,'FTEN' ,'CBOE' ,'CIT' ,'PBR' ,'HBC' ,'FDS' ,'BMO' ,'MORN' ,'MUDS' ,'FRC' ,'CG' ,'AMG' ,'PBCP' ,'MAAC' ,'BKLY' ,'WAL' ,'FLGS' ,'EBC' ,'FCNCA' ,'DB' ,'TW' ,'TREE' ,'IBKR' ,'XP' ,'FHI' ,'FMBI' ,'BNS' ,'ORI' ,'FNT' ,'ING' ,'MFC' ,'SF' ,'VOYA' ,'CFR' ,'LTR' ,'BPOP' ,'ARCC' ,'EWBC' ,'ARES' ,'VIRT' ,'EVR' ,'CSR' ,'RCHI' ,'LAZ' ,'MKL' ,'WPF' ,'TCBI' ,'TMK' ,'ASB' ,'UNM' ,'TRUP' ,'ISBC' ,'UMPQ' ,'APAM' ,'WBST' ,'PNFP' ,'LUK' ,'JHG' ,'CMT' ,'PSTH' ,'RGA' ,'AFG' ,'PRSP' ,'GNW' ,'FCBP' ,'SEIC' ,'SNV' ,'GHIV' ,'BCS' ,'BCM' ,'Y' ,'AGC' ,'ESPD' ,'UBS' ,'UNIT' ,'GSHD' ,'PFSI' ,'FAF' ,'BXS' ,'AIZ' ,'NAVIV' ,'AFC' ,'CHFC' ,'ABCB' ,'UMBF' ,'CNO' ,'AXS' ,'WD' ,'GBCI' ,'COWND' ,'BPBC' ,'CONX' ,'LOB' ,'SLC' ,'PCI' ,'VLY' ,'LC' ,'PHYS' ,'FBAN' ,'OZRK' ,'BOH' ,'FULT' ,'HBHC' ,'CIB' ,'FNC' ,'STD' ,'ORCC' ,'FBP' ,'CBU' ,'FRHC' ,'SC' ,'DGNR' ,'INDB' ,'HLI' ,'PSEC' ,'EHTH' ,'PJT' ,'TBK' ,'SLQT' ,'PAWN' ,'BHFWV' ,'AC' ,'RILY' ,'FOCS' ,'FFIN' ,'AGO' ,'WTFC' ,'SEAH' ,'CADE' ,'BKU' ,'BOKF' ,'AAC' ,'GDOT' ,'KNSL' ,'CBSH' ,'ARC' ,'ESNT' ,'FWAA' ,'CATY' ,'NMIH' ,'FFC' ,'MC' ,'EBC' ,'VRTS' ,'RLI' ,'FHB' ,'PLMR' ,'MAIN' ,'PDI' ,'PRI' ,'TIRX' ,'SFNCA' ,'LPRO' ,'ENVA' ,'UBSI' ,'HTGC' ,'CASH' ,'ROOT' ,'HLNE' ,'EBSB' ,'GSBD' ,'LFCO' ,'EFC' ,'PHC' ,'PTY' ,'FTCV' ,'ERIE' ,'ATCO' ,'COLB' ,'UTF' ,'BBV' ,'CEF' ,'STC' ,'SAN' ,'MCY' ,'GOCO' ,'CFAC' ,'RQI' ,'EFSC' ,'GMII' ,'AACQ' ,'SPFR' ,'SIGI' ,'THCB' ,'CNS' ,'RTP' ,'FCBP' ,'OMAM' ,'CUBI' ,'IBTX' ,'SRNG' ,'LDI' ,'PRAA' ,'AEL' ,'WFSL' ,'PIPR' ,'SFBS' ,'GWB' ,'HOMB' ,'LFC' ,'INDI' ,'ESGRD' ,'AJAX' ,'SHG' ,'MCMC' ,'TYP' ,'WSFS' ,'CVII' ,'CRHC' ,'GHL' ,'NARA' ,'PFS' ,'IPOF' ,'CLM' ,'BOFI' ,'GOF' ,'SPRQ' ,'NAD' ,'PXT' ,'CNA' ,'CNBC' ,'OFG' ,'HEC' ,'GBDC' ,'UCBI' ,'FRME' ,'EIG' ,'DFIN' ,'ACTC' ,'NSL' ,'GRSV' ,'FRX' ,'FSC' ,'BTWN' ,'MGI' ,'OLDB' ,'TRMK' ,'SPAQ' ,'DHY' ,'BDGE' ,'MUFG' ,'WSBC' ,'FFBC' ,'GFX' ,'APXT' ,'FWWB' ,'PDO' ,'EQOS' ,'HYT' ,'UBSH' ,'FAMCK' ,'NVG' ,'APGB' ,'TOWN' ,'FTOC' ,'PDAC' ,'SASR' ,'SBCFA' ,'VGAC' ,'EGBN' ,'BCX' ,'GCMG' ,'BDJ' ,'NBHC' ,'BHL' ,'UFCS' ,'HFRO' ,'JQC' ,'HZAC' ,'SMFG' ,'PUCK' ,'VBTX' ,'FMAC' ,'EXG' ,'MAACU' ,'LGAC' ,'CLII' ,'FVH' ,'BGY' ,'BMTC' ,'RA' ,'DNP' ,'EIS' ,'TBA' ,'JRVR' ,'OXLC' ,'OCFC' ,'LCY' ,'HTLF' ,'GSAH' ,'HAFC' ,'TPGY' ,'SCU' ,'NWSB' ,'JWS' ,'NEA' ,'USA' ,'NMFC' ,'FIBK' ,'BANF' ,'ACIC' ,'LYG' ,'BSBR' ,'UTG' ,'FBF' ,'AEG' ,'KB' ,'BOWX' ,'FPTB' ,'FCF' ,'OLA' ,'HMN' ,'AMSF' ,'CVB' ,'SII' ,'GHVI' ,'LUXA' ,'GROW' ,'TREB' ,'NTB' ,'PFN' ,'LKFN' ,'LBAI' ,'FSIC' ,'SFTW' ,'TSLX' ,'GLEO' ,'SRAC' ,'EVT' ,'SNPR' ,'IBOC' ,'FSKR' ,'HOL' ,'PACE' ,'SRCE' ,'BSTZ' ,'FBK' ,'WABC' ,'TBBKD' ,'EVV' ,'RIV' ,'CRVL' ,'FUSE' ,'OCA' ,'BMEZ' ,'BRP' ,'FBMS' ,'RVT']
RealEstate = ['AMT' ,'SPG' ,'TWRS' ,'EQIXD' ,'O' ,'AMB' ,'AHT' ,'DLR' ,'ARCP' ,'WY' ,'VICI' ,'KRC' ,'RIGX' ,'HCN' ,'PSA' ,'OPEN' ,'NLY' ,'ARE' ,'SBAC' ,'CBRE' ,'DRE' ,'KIM' ,'EQR' ,'PEAK' ,'VC' ,'CDC' ,'EXR' ,'LAMR' ,'BYA' ,'MAA' ,'RDFN' ,'AGNC' ,'VNO' ,'ESS' ,'INVH' ,'IMTN' ,'MPW' ,'BXP' ,'OHI' ,'CPT' ,'CONE' ,'LAP' ,'COLD' ,'SUI' ,'PK' ,'STWD' ,'NRZ' ,'UDR' ,'DEI' ,'CBSO' ,'YSI' ,'GLPIV' ,'CT' ,'NNN' ,'REG' ,'GET' ,'AMH' ,'HTA' ,'FRT' ,'COR' ,'REXR' ,'APLE' ,'PEB' ,'WRI' ,'CUZ' ,'STOR' ,'LSI' ,'FR' ,'BRX' ,'SLG' ,'EXPI' ,'EST' ,'MHC' ,'IIPR' ,'DOC' ,'MAC' ,'STIR' ,'ADC' ,'SRC' ,'EGP' ,'EPR' ,'HPP' ,'SUNH' ,'PCH' ,'ACC' ,'HIW' ,'BNL' ,'HRP' ,'PZN' ,'DDR' ,'CWK' ,'MCT' ,'MGP' ,'NAQ' ,'RLGY' ,'EPRT' ,'RYN' ,'RLJ' ,'BDN' ,'NHI' ,'QTS' ,'COPT' ,'TRNO' ,'SKT' ,'SHO' ,'CIM' ,'WHC' ,'ESRT' ,'UE' ,'BPY' ,'HR' ,'FCPT' ,'HHC' ,'AIRC' ,'ABR' ,'HASI' ,'IVR' ,'RPAI' ,'DRH' ,'SVC' ,'ORC' ,'AIV' ,'PGRE' ,'IRT' ,'JBGS' ,'TWO' ,'CTREV' ,'LXP' ,'CLNY' ,'NTAIF' ,'AAT' ,'XHR' ,'AHP' ,'DEA' ,'LTC' ,'CSALV' ,'BPR' ,'ARI' ,'CLI' ,'PMT' ,'MFA' ,'GNL' ,'RWTI' ,'PSM' ,'NSA' ,'APT' ,'SJP' ,'NTR' ,'DX' ,'CXP' ,'WRE' ,'KREF' ,'KRG' ,'PDM' ,'GMRE' ,'HT' ,'FSV' ,'NMRK' ,'CHCT' ,'FSRVF' ,'ACRE' ,'FTHM' ,'BRMK' ,'GBR' ,'GOOD' ,'LADR' ,'SAFE' ,'ALEX' ,'ILPT' ,'SRG' ,'ALX' ,'GOV' ,'PSTL' ,'NTST' ,'RMAX' ,'VINO' ,'LAND' ,'UMH' ,'PAX' ,'INN' ,'GTY']
ConsumerDefensive = ['EL' ,'WMT' ,'PG' ,'COST' ,'PEP' ,'CLX' ,'KO' ,'KFT' ,'DH' ,'BYND' ,'PM' ,'CL' ,'MO' ,'CHGG' ,'KR' ,'DG' ,'KMB' ,'KHC' ,'CBRNA' ,'HSY' ,'ADM' ,'GIS' ,'TSN' ,'SYY' ,'HANS' ,'BG' ,'ACCOB' ,'DLTR' ,'K' ,'NWL' ,'DPS' ,'CAG' ,'EDU' ,'BTI' ,'FAMI' ,'SAM' ,'LW' ,'OLLI' ,'USFD' ,'MCCRK' ,'DEO' ,'CPB' ,'UNFI' ,'SJMA' ,'DAR' ,'HRL' ,'FRPT' ,'CHD' ,'BUD' ,'UL' ,'COTY' ,'THS' ,'BJ' ,'BFB' ,'SFM' ,'PFGC' ,'ABEV' ,'TWOU' ,'GO' ,'HLF' ,'CPO' ,'FMCI' ,'CNS' ,'HELE' ,'NOMD' ,'SPB' ,'COUR' ,'VFF' ,'ACI' ,'LANC' ,'PPC' ,'LAUR' ,'CASY' ,'SAFM' ,'HAIN' ,'GRSH' ,'PDA' ,'DV' ,'NUS' ,'POST' ,'UTZ' ,'PRMW' ,'LOPE' ,'NOVS' ,'STRA' ,'CELH' ,'FMX' ,'CCEP' ,'ELF' ,'EPC' ,'BGS' ,'STKLF' ,'FIZ' ,'TPB' ,'WPO' ,'JJSF' ,'CENTA' ,'USNA' ,'OAC' ,'SMPL' ,'LRN' ,'BGL' ,'FLO' ,'CHEF' ,'NTCO' ,'HMHC' ,'METX' ,'CECO' ,'KOF' ,'COKE' ,'EEIQ' ,'PSMT' ,'ARCE' ,'JEAN' ,'TR' ,'IMKTA' ,'CORE' ,'ANDE' ,'WAFU' ,'UVV' ,'SPTN' ,'AFYA' ,'NAPA' ,'CVGW' ,'WVVI' ,'WMK' ,'CCUUY' ,'CALM']
Healthcare = ['HSGX' ,'MRNA' ,'NOX' ,'PFE' ,'PRPO' ,'JNJ' ,'RHE' ,'UNH' ,'AMGN' ,'ABBV' ,'MRK' ,'TDOC' ,'CVS' ,'TMO' ,'GILD' ,'LLY' ,'NABI' ,'BMY' ,'MDT' ,'TLRY' ,'ALGN' ,'ABT' ,'DHR' ,'ALXN' ,'ATH' ,'ISRG' ,'BSX' ,'ZTS' ,'REGN' ,'IDPH' ,'CI' ,'HCA' ,'HUM' ,'BDX' ,'NVS' ,'SYK' ,'RESM' ,'VRTX' ,'IDXX' ,'ZEN' ,'AVII' ,'CNTE' ,'EW' ,'VEEV' ,'DXCM' ,'WST' ,'WAG' ,'HOLX' ,'ILMN' ,'ZMH' ,'CERN' ,'CRSP' ,'IQV' ,'WAT' ,'AAS' ,'BAX' ,'HSIC' ,'MCK' ,'DGX' ,'SNDL' ,'LH' ,'EXAS' ,'CVAC' ,'CAH' ,'IPOC' ,'EGG' ,'A' ,'CRL' ,'ELAN' ,'PACB' ,'CTLT' ,'STRL' ,'GLX' ,'PPD' ,'QDEL' ,'JAZZ' ,'BNGO' ,'ABMD' ,'HZNP' ,'TFX' ,'BVF' ,'SGEN' ,'TXG' ,'NVO' ,'MRTX' ,'TEVIY' ,'NVTA' ,'NVCR' ,'AMWL' ,'THC' ,'TRL' ,'AMED' ,'BIOA' ,'CGC' ,'MNMD' ,'MTD' ,'XRAY' ,'ICLRY' ,'COO' ,'FLGT' ,'MOH' ,'GH' ,'PODD' ,'NBIX' ,'TWST' ,'GDRX' ,'ALNY' ,'STAA' ,'BGNE' ,'UHS' ,'PEN' ,'NTLA' ,'VTRSV' ,'AUPH' ,'NTRA' ,'KRG' ,'NVRO' ,'MASI' ,'BEAM' ,'TECH' ,'UTHR' ,'RPRX' ,'INCY' ,'PTIE' ,'ZLAB' ,'ISIP' ,'BMRN' ,'SNY' ,'IOVA' ,'PRAH' ,'HB' ,'INCR' ,'TGTX' ,'NARI' ,'NVST' ,'QGENF' ,'FATE' ,'AMS' ,'RGEN' ,'CHE' ,'PTCT' ,'CTXR' ,'ADPT' ,'SWAV' ,'ZOM' ,'EBS' ,'PHG' ,'TNDM' ,'CHNG' ,'OMCL' ,'HCAT' ,'HRC' ,'OSH' ,'SDGR' ,'NNOX' ,'GEB' ,'PRGO' ,'ACHC' ,'FOLD' ,'ALC' ,'MRVI' ,'ACCD' ,'CYH' ,'GMAB' ,'CNMD' ,'NUVA' ,'HGEN' ,'KBIO' ,'OCUL' ,'SRNE' ,'HTI' ,'HAE' ,'EDIT' ,'EXEL' ,'ARNA' ,'ONEM' ,'AXNX' ,'BHVN' ,'ACAD' ,'ACB' ,'ARWR' ,'TPTX' ,'RAD' ,'CNTL' ,'PETS' ,'PRVA' ,'INSP' ,'CNAC' ,'PGNY' ,'NSTG' ,'RARE' ,'SAGE' ,'TAK' ,'IART' ,'GKOS' ,'BBIO' ,'RETA' ,'AVIR' ,'KPTI' ,'INMD' ,'AAVL' ,'OMI' ,'APLS' ,'IRTC' ,'AHS' ,'ARQT' ,'ARGX' ,'AGIO' ,'CARA' ,'ALKS' ,'VCYT' ,'LMNX' ,'QTRX' ,'CDNA' ,'PAVM' ,'KALA' ,'ATRC' ,'AXSM' ,'SHC' ,'CCXI' ,'ATOS' ,'MGNX' ,'TBIO' ,'PVTR' ,'SILK' ,'XLRN' ,'MEDP' ,'BCRX' ,'MGLN' ,'ACRS' ,'GMED' ,'ARCT' ,'CRON' ,'KROS' ,'KOD' ,'PINC' ,'LUNG' ,'XXII' ,'GBT' ,'AHCO' ,'ENSG' ,'LHCG' ,'VIR' ,'ICUI' ,'CYPH' ,'SEEL' ,'EBIO' ,'ARVN' ,'ARDX' ,'SDC' ,'TRIL' ,'OGI' ,'SIBN' ,'MDRX' ,'HOTR' ,'LGND' ,'LIVN' ,'PDX' ,'HIV' ,'INSM' ,'PCRX' ,'INHL' ,'HQY' ,'BLUE' ,'AGL' ,'INOV' ,'BPMC' ,'ALLO' ,'DNLI' ,'ZNTL' ,'SENS' ,'AFMD' ,'BDAL' ,'PHR' ,'AHPA' ,'SEER' ,'CLVS' ,'DRNA' ,'PDCO' ,'CRY' ,'HEXO' ,'CVETV' ,'KIDS' ,'ALDX' ,'NEO' ,'CYTK' ,'DVAX' ,'FGEN' ,'MYGN' ,'CTMX' ,'EAR' ,'TCMD' ,'IRWD' ,'BLI' ,'SEM' ,'ABCL' ,'ASTM' ,'CRIS' ,'GLPG' ,'SANA' ,'SNN' ,'PETQ' ,'KYMR' ,'CHRS' ,'OM' ,'ALLK' ,'GTHX' ,'MOTS' ,'RLAY' ,'TCLN' ,'ITCI' ,'LNTH' ,'PTBI' ,'CDXS' ,'ADUS' ,'ICPT' ,'AVAH' ,'GOVX' ,'RUBY' ,'PSNL' ,'ITEK' ,'SURF' ,'QURE' ,'RCM' ,'OCDX' ,'MMSI' ,'HOTH' ,'ESPR' ,'HCSG' ,'RVNC' ,'GB' ,'CPRX' ,'RDY' ,'HNGR' ,'ENDP' ,'ADMA' ,'HSAC' ,'GRFS' ,'KITS' ,'ALEC' ,'ALT' ,'CCXX' ,'GTBP' ,'ZYME' ,'ALXO' ,'TCEL' ,'SUPN' ,'PRTA' ,'IMGN' ,'EVH' ,'KURA' ,'CNST' ,'CNDO' ,'GBS' ,'RVMD' ,'BTAI' ,'REPL' ,'INGN' ,'GBIO' ,'SGRY' ,'RGNX' ,'AMTI' ,'HSKA' ,'HYH' ,'CERS' ,'SNDX' ,'NEOG' ,'TMDX' ,'CODX' ,'CSTL' ,'VREX' ,'LMAT' ,'OPCH' ,'RGLS' ,'HRTX' ,'PLCC' ,'FLDM' ,'SELB' ,'SGMO' ,'SIMU' ,'AVRO' ,'FREQ' ,'MDGL' ,'IGMS' ,'MCRB' ,'RCUS' ,'CERT' ,'ZGNX' ,'CORT' ,'DCPH' ,'AKRO' ,'MNKD' ,'CRTX' ,'PRSC' ,'FHC' ,'EPZM' ,'PTGX' ,'KRTX' ,'MASS' ,'FMS' ,'GALT' ,'PBH' ,'JAGX' ,'NEOT' ,'VSTM' ,'EYES' ,'MYOV' ,'PLX' ,'ADCT' ,'USPH' ,'HOWL' ,'ADTX' ,'CATS' ,'KDMN' ,'FMTX' ,'STIM' ,'YMAB' ,'TXMD' ,'PRLD' ,'CBYL' ,'TRHC' ,'TIL' ,'EPTO' ,'ATHA' ,'AVXL' ,'NRIX' ,'PRQR' ,'AMRX' ,'ANGO' ,'SRRK' ,'MLSS' ,'GRTX' ,'TVTX' ,'JYNT' ,'TRVN' ,'BKD' ,'OPHT' ,'BDTX' ,'RIGL' ,'AKBA' ,'MTBCP' ,'IVCR' ,'TNXP' ,'ONTX' ,'APTO' ,'BABY' ,'RPTX' ,'ZIOP' ,'AMHC' ,'GTXI' ,'EWTX' ,'AGEN' ,'PRAX' ,'ATEC' ,'XNCR' ,'EPIX' ,'MRSN' ,'PHAT' ,'OPRX' ,'KRYS' ,'BCTX' ,'RDUS' ,'ATRI' ,'CGEN' ,'RDYN' ,'ITOS' ,'HARP' ,'WVE' ,'ITRM' ,'XERS' ,'AERI' ,'ENTA' ,'MORF' ,'TCRR' ,'ATRA' ,'RXRX' ,'BCAB' ,'XON' ,'SWTX' ,'CCCC' ,'NOVN' ,'EOLS' ,'VNDA' ,'SGFY' ,'FLXN' ,'PAHC' ,'VKTX' ,'KBLM' ,'STTK' ,'ESTA' ,'PRVB' ,'ATNX' ,'ZYXI' ,'NEOT' ,'BLFS' ,'AVEO' ,'OSCR' ,'SPNEV' ,'EVLO' ,'CRDF' ,'RYTM' ,'ORMP' ,'SBTX' ,'BIOD' ,'ALHC' ,'NKTX' ,'VTGN' ,'PMVP' ,'OMER']
Utilities = ['FPL' ,'DUK' ,'D' ,'FE' ,'PCG' ,'PE' ,'SO' ,'AEP' ,'DTE' ,'AES' ,'EIX' ,'WEC' ,'NU' ,'PEG' ,'PPL' ,'NSP' ,'ED' ,'SRE' ,'CMS' ,'NI' ,'AEE' ,'ETR' ,'WPH' ,'AWK' ,'PNW' ,'ATO' ,'HOU' ,'NRG' ,'VST' ,'WR' ,'SJI' ,'OGE' ,'ORA' ,'BEPC' ,'BEPCW' ,'IDA' ,'PCGU' ,'WWP' ,'ABY' ,'WTRG' ,'UGI' ,'POR' ,'SWX' ,'UTL' ,'OGS' ,'CWEN' ,'PNM' ,'LG' ,'NEP' ,'BKH' ,'NJR' ,'NWEC' ,'AGR' ,'SBS' ,'BEP' ,'NGG' ,'ENI' ,'FTS' ,'ELP' ,'HE' ,'MPL' ,'NWNG' ,'AQN' ,'SCW' ,'CWT' ,'CIG' ,'NFE' ,'BIP' ,'BIPC' ,'AZRE' ,'SJW' ,'CPK' ,'AEPPZ' ,'OTTR']
CommunicationServices = ['FB' ,'TWTR' ,'NFLX' ,'GOOCV' ,'GOOG' ,'DIS' ,'VIAC' ,'BIDU' ,'BEL' ,'PINS' ,'ROKU' ,'CMCSA' ,'SNAP' ,'ZM' ,'ATVI' ,'SBC' ,'PCS' ,'ABNB' ,'TWLO' ,'DISCA' ,'MTCH' ,'SPOT' ,'Z' ,'BILI' ,'CHTR' ,'FEAC' ,'AMC' ,'ERTS' ,'EDR' ,'RBLX' ,'DISCK' ,'TTWO' ,'FUBO' ,'DASH' ,'FOXAV' ,'IAC' ,'ZNGA' ,'OMC' ,'IPG' ,'FVRR' ,'DISH' ,'CTL' ,'LBDKV' ,'NYT' ,'ATUS' ,'CDRD' ,'NTES' ,'NXST' ,'GCSB' ,'GRUB' ,'NWSAV' ,'SSTK' ,'LYV' ,'RUBI' ,'MDP' ,'ZG' ,'CABO' ,'AIX' ,'WMG' ,'CNK' ,'YNDX' ,'VOD' ,'ATHM' ,'CDLX' ,'BCE' ,'TDS' ,'AMCX' ,'AMX' ,'GRPN' ,'WWFE' ,'FOXBV' ,'SBGI' ,'MSGS' ,'LBTYA' ,'LSXMA' ,'GNUS' ,'LSXMK' ,'LMCKV' ,'IHRT' ,'YELP' ,'GHQ' ,'HOFV' ,'TU' ,'LBTYK' ,'COI' ,'ORBC' ,'GCI' ,'MAIL' ,'VG' ,'JWA' ,'TV' ,'IMAXF' ,'GOGO' ,'DLX' ,'TEF' ,'NWSVV' ,'IDC' ,'CARG' ,'COUP' ,'MSGE' ,'TTGT' ,'MSGEW' ,'RG' ,'SAQN' ,'LBDAV' ,'WPP' ,'PLTK' ,'SCR' ,'GENI' ,'SCPL' ,'MBT' ,'VIP' ,'EVER' ,'RUK' ,'DLPN' ,'LILKV' ,'SHET' ,'AVID' ,'INTN' ,'SJR' ,'DMD' ,'WIFI' ,'SSP' ,'VPRT' ,'MSG' ,'USM' ,'MCS' ,'ADV' ,'IZEA' ,'KTC' ,'FTE' ,'LGF.B' ,'CNSL' ,'EROS' ,'CCO' ,'LGF.A' ,'VIV' ,'BRIOF' ,'BOMN' ,'NGMS' ,'QNST']
Energy = ['XON' ,'CHV' ,'P' ,'MRO' ,'KMI' ,'VLO' ,'SLB' ,'MPC' ,'PBR' ,'EOG' ,'DVN' ,'PSX' ,'BHGE' ,'OXY' ,'HAL' ,'BP' ,'PXD' ,'RDSA' ,'COG' ,'WMB' ,'EPD' ,'CCJ' ,'FANG' ,'SU' ,'AHC' ,'OKE' ,'APA' ,'IPPIF' ,'TOT' ,'RDSB' ,'CHEX' ,'PBF' ,'NOI' ,'CED' ,'REGI' ,'AR' ,'PBRA' ,'MPLX' ,'WEG' ,'LOM' ,'TPL' ,'TRGP' ,'ETE' ,'MARY' ,'HOC' ,'SWN' ,'OVV' ,'EQT' ,'TRP' ,'DEN' ,'FTI' ,'RIG' ,'DNN' ,'IMO' ,'CLR' ,'VVV' ,'CVE' ,'SLCA' ,'MUR' ,'MTDR' ,'CHK' ,'CLNP' ,'ETRN' ,'HP' ,'SRAQ' ,'WLL' ,'STO' ,'XEC' ,'CNX' ,'CLNE' ,'AMGP' ,'BTU' ,'APY' ,'UUUU' ,'TPGE' ,'ARCH' ,'PSXP' ,'STNG' ,'NXE' ,'TRCH' ,'UEC' ,'BRN' ,'PETD' ,'SHLX' ,'GLNG' ,'NFG' ,'FRO' ,'EC' ,'TS' ,'OAS' ,'PAA' ,'DK' ,'PBA' ,'SUSP' ,'LPI' ,'BCEI' ,'AMTX' ,'DHT' ,'PUMP' ,'PTEN' ,'CRK' ,'NBR' ,'PAGP' ,'VET' ,'CVI' ,'KOS' ,'INT' ,'WHD' ,'UDL' ,'MPET' ,'LBRT' ,'NRGY' ,'CPG' ,'OII' ,'NEX' ,'DRQ' ,'KAAC' ,'WGP' ,'EURN' ,'ENLC' ,'HMLP' ,'ERF' ,'NOG' ,'SASOY' ,'ENBL' ,'PTR' ,'GLP' ,'CRLBF' ,'TNK' ,'URG' ,'E' ,'HEP' ,'DPM' ,'VNOM' ,'EXH' ,'OIS' ,'PVAC' ,'CQP' ,'CEIX' ,'DNOW' ,'SGY' ,'LPG' ,'CDIS' ,'FI' ,'RES' ,'WTI' ,'UGP' ,'MRC' ,'SNP' ,'GEL' ,'SOI' ,'BOOM' ,'TTI']
Industrials = ['BA' ,'FDX' ,'UPS' ,'GE' ,'PLUG' ,'CAT' ,'AAL' ,'UNP' ,'DAL' ,'UTX' ,'LMT' ,'ALD' ,'UAUA' ,'SWK' ,'NOC' ,'AUD' ,'MRKT' ,'GPN' ,'MMM' ,'PH' ,'LUV' ,'JCI' ,'SPCE' ,'ETN' ,'CSX' ,'ITW' ,'EMR' ,'CUM' ,'DE' ,'ROK' ,'HRS' ,'ERC' ,'NSC' ,'SZI' ,'J' ,'EFX' ,'UW' ,'CD' ,'OTIS' ,'OTISW' ,'CARR' ,'GNRC' ,'WAB' ,'CARRW' ,'JBLU' ,'SAVE' ,'CP' ,'ALLE' ,'LPX' ,'CNI' ,'ODFL' ,'AME' ,'ROP' ,'FAST' ,'GD' ,'URI' ,'IR' ,'TDG' ,'KSU' ,'EXPD' ,'AVY' ,'PCAR' ,'BLDPF' ,'CHRW' ,'VRSK' ,'PAYX' ,'JBHT' ,'AOS' ,'TXT' ,'GWW' ,'XYL' ,'OC' ,'CTAS' ,'AG' ,'RSG' ,'WCNX' ,'TRU' ,'MAS' ,'SBLK' ,'IEX' ,'SNA' ,'BLDR' ,'CFX' ,'PWR' ,'ALK' ,'DOV' ,'UPWK' ,'VRT' ,'OTRKB' ,'MIDD' ,'BLD' ,'CPRT' ,'BE' ,'ATKR' ,'EMCG' ,'DAC' ,'EAF' ,'NLSN' ,'AA' ,'WBT' ,'IAA' ,'MTZ' ,'LII' ,'TWP' ,'AER' ,'IIN' ,'KNGT' ,'HUBB' ,'DGLY' ,'ZIM' ,'RBC' ,'RHI' ,'BAH' ,'HII' ,'SPR' ,'R' ,'TOPT' ,'TKR' ,'AIMC' ,'NDSN' ,'TEX' ,'TOC' ,'GGG' ,'GY' ,'EAGL' ,'WSO' ,'SRG' ,'ZTO' ,'SWBI' ,'CSL' ,'ARNC' ,'TFII' ,'NMM' ,'WMS' ,'AZEK' ,'ACM' ,'PRLB' ,'PRG' ,'AYI' ,'WGOV' ,'FTIC' ,'HYFM' ,'WATR' ,'LSTR' ,'KMT' ,'SRCL' ,'MSM' ,'TPIC' ,'KBR' ,'FLS' ,'ERJ' ,'SITE' ,'KEX' ,'CNHI' ,'TTC' ,'FLR' ,'VALM' ,'PNR' ,'ADT' ,'ALT' ,'SCST' ,'ABB' ,'ASF' ,'PZB' ,'PBI' ,'IBP' ,'DY' ,'RCII' ,'AAN' ,'DOOR' ,'RYAAY' ,'AWI' ,'WFII' ,'GFL' ,'LAND' ,'WTHG' ,'ABM' ,'DCI' ,'GSHT' ,'HAYW' ,'AAWW' ,'ALGT' ,'UHAL' ,'MMS' ,'TASR' ,'SSD' ,'CMCO' ,'TSP' ,'RAIL' ,'DPW' ,'MAN' ,'REZI' ,'WCC' ,'RGR' ,'HEI' ,'TAL' ,'POWW' ,'AQUA' ,'ROCK' ,'KNL' ,'NVT' ,'MIC' ,'CPA' ,'AMRC' ,'ENR' ,'CW' ,'MNES' ,'ASGN' ,'SB' ,'BWC' ,'TNET' ,'FTAI' ,'RBA' ,'BECN' ,'CYRX' ,'DHC' ,'LECO' ,'AVAV' ,'GVA' ,'HTZ' ,'PSN' ,'WERN' ,'AL' ,'GLBS' ,'HUBG' ,'NAV' ,'CR' ,'ABFS' ,'GSTL' ,'HXL' ,'RXN' ,'VLCCF' ,'GBX' ,'JELD' ,'CATM' ,'ENS' ,'YGMZ' ,'CRS' ,'SKYW' ,'ALEX' ,'FELE' ,'MWA' ,'FRTA' ,'ACA' ,'JBT' ,'TRN' ,'AZUL' ,'OMAB' ,'ROLL' ,'HA' ,'AEIS' ,'FIX' ,'ACP' ,'WTS' ,'B' ,'MEG' ,'FAIL' ,'BWC' ,'TGH' ,'ICFI' ,'HEIA' ,'GMS' ,'UNF' ,'BLDG' ,'SFL' ,'PGTI' ,'SNDR' ,'AZZ' ,'EGLE' ,'HSII' ,'APG' ,'SWIM' ,'MAQ' ,'KAMNA' ,'KRNT' ,'HURN' ,'DSX' ,'ULCC' ,'CMRE' ,'CLHB' ,'ABXA' ,'ENRJ' ,'CBIZ' ,'MRCY' ,'GOL' ,'HONI' ,'GMT' ,'BMI' ,'AIR' ,'SPW' ,'MLI' ,'CGT' ,'APZ' ,'FSS' ,'HI' ,'PRIM' ,'DNY' ,'PIXY' ,'KFY' ,'LNN' ,'<NULL>' ,'ECHO' ,'MOG.A' ,'XONE' ,'KODK' ,'GLDD' ,'NAT' ,'APOG' ,'MYRG' ,'CWST' ,'BXC' ,'HEES' ,'EPAC' ,'SHYF' ,'SMED' ,'RADIF' ,'MTW' ,'ROMC' ,'JOB' ,'HSC' ,'VRTV' ,'ECOL' ,'CAP' ,'ERII' ,'SNCY' ,'REVG' ,'BRCOA' ,'AAON' ,'SNHY' ,'TANT' ,'ROAD' ,'HYRE' ,'BULK' ,'TWI' ,'CBAK' ,'MIII' ,'LBOR' ,'NPO' ,'BBCP' ,'MATW' ,'LDL' ,'NVEE' ,'INDU' ,'FLOW' ,'SINO' ,'BW' ,'MGRC' ,'ESEA' ,'TFT' ,'VSEC' ,'GFLU' ,'VTSI' ,'SCS' ,'HTLD' ,'WIRE' ,'GFF' ,'CSWIV' ,'PCR' ,'TGI' ,'GRSHU' ,'RAVN' ,'MESA' ,'ASPN' ,'VVI' ,'TRS' ,'ASTE' ,'EOSE']
Technology = ['AAPL' ,'MSFT' ,'AMD' ,'NVDA' ,'SHOP' ,'CRM' ,'MU' ,'SQ' ,'INTC' ,'NOW' ,'FISV' ,'ORCL' ,'AMAT' ,'QCOM' ,'TXN' ,'COIN' ,'TSM' ,'IBM' ,'UBER' ,'LRCX' ,'PFPT' ,'AVGO' ,'ADBE' ,'CSCO' ,'PLTR' ,'AHA' ,'APPY' ,'OKTA' ,'ENPH' ,'SNOW' ,'KLAC' ,'NXPI' ,'ISNS' ,'CRWD' ,'DOCU' ,'CEY' ,'ADI' ,'NET' ,'LYFT' ,'DDOG' ,'TTD' ,'ON' ,'MVIS' ,'ASMLF' ,'FSLR' ,'NOK' ,'XLNX' ,'MRVL' ,'STX' ,'TEAM' ,'WDAY' ,'ZS' ,'CTSH' ,'CHKPF' ,'WDC' ,'RNG' ,'INTU' ,'VSNR' ,'HUBS' ,'SEDG' ,'GLW' ,'U' ,'MSTR' ,'MCHP' ,'FTNT' ,'FSLY' ,'CTXS' ,'PANW' ,'COUP' ,'MNDL' ,'APH' ,'ZBRA' ,'DELL' ,'AI' ,'EPAM' ,'QRVO' ,'ACN' ,'MPWR' ,'CREE' ,'AKAM' ,'HWP' ,'FFIV' ,'FLT' ,'PMTC' ,'ADSK' ,'PATH' ,'RUN' ,'SPLK' ,'MXIM' ,'WORK' ,'IIVI' ,'KEYS' ,'ZEN' ,'FIVN' ,'GDS' ,'CDN' ,'TDY' ,'MDB' ,'TEL' ,'WIX' ,'SPWR' ,'TER' ,'VMW' ,'FTV' ,'ENTG' ,'MOT' ,'VRSN' ,'BILL' ,'TYL' ,'CDW' ,'ANET' ,'ESTC' ,'STNE' ,'TRMB' ,'SNPS' ,'HPE' ,'BR' ,'VUZI' ,'JNPR' ,'AFRM' ,'DBX' ,'IRBT' ,'LOGIY' ,'SYMC' ,'PAYC' ,'NTAP' ,'GART' ,'SAP' ,'SAI' ,'AVLR' ,'CRUS' ,'JKHY' ,'G' ,'IPGP' ,'WXS' ,'DLB' ,'EVBG' ,'LITEV' ,'PLAN' ,'PAGS' ,'LSCC' ,'SSNC' ,'UMC' ,'BOX' ,'AYX' ,'ANSS' ,'GMHI' ,'LAZR' ,'BMBL' ,'AZPN' ,'NCR' ,'SAIL' ,'FEYE' ,'INFY' ,'GDDY' ,'CDAY' ,'CACI' ,'ERICY' ,'GRMN' ,'SABR' ,'STM' ,'ST' ,'ZI' ,'SMAR' ,'PANL' ,'LPSN' ,'APPN' ,'BRKS' ,'DT' ,'COHR' ,'VRNS' ,'FLEXF' ,'DXC' ,'NCNO' ,'QTWO' ,'CRSR' ,'EEFT' ,'FOUR' ,'CGNX' ,'SONO' ,'RIMM' ,'RPD' ,'QLYS' ,'NOVA' ,'KLIC' ,'SYNA' ,'FIC' ,'MDA' ,'BAND' ,'JBIL' ,'NICEY' ,'GWRE' ,'FAF' ,'BKI' ,'ARW' ,'MANH' ,'EBON' ,'BIGC' ,'CSIQ' ,'CCC' ,'TDSC' ,'SLAB' ,'CIEN' ,'XM' ,'PSTG' ,'XRX' ,'SHLS' ,'NATI' ,'NTNX' ,'EGHT' ,'TDC' ,'AMKR' ,'PD' ,'MKSI' ,'VSH' ,'GLOB' ,'MTSI' ,'PCTY' ,'VZIO' ,'AMBA' ,'NEWR' ,'LSPD' ,'UCTT' ,'VSAT' ,'JFAX' ,'VRNT' ,'SGH' ,'ITRI' ,'CYBR' ,'DOX' ,'TENB' ,'VLDR' ,'SPT' ,'CLDR' ,'COMM' ,'COHU' ,'VNT' ,'VNTW' ,'PI' ,'BL' ,'STMP' ,'FROG' ,'CRNCV' ,'PEGA' ,'EDA' ,'CCMP' ,'UNPH' ,'MCFE' ,'CDKVV' ,'MX' ,'BSY' ,'ALTR' ,'YOD' ,'WK' ,'MDLA' ,'KOPN' ,'SWCH' ,'ENV' ,'INFN' ,'NTGR' ,'TSAI' ,'GPRO' ,'SSYS' ,'DIO' ,'ARRY' ,'EXTR' ,'FORM' ,'ALRM' ,'TRNE' ,'DNB' ,'CALX' ,'OTEXF' ,'BLKB' ,'PTAC' ,'DLPH' ,'CLSK' ,'POWI' ,'AVYA' ,'PAMT' ,'MIME' ,'SAIC' ,'CVLT' ,'AVT' ,'MOGO' ,'GILTF' ,'EB' ,'ASAN' ,'LFUS' ,'RXT' ,'APPF' ,'UBNT' ,'ACXM' ,'PING' ,'SNX' ,'SWI' ,'RMBS' ,'NSIT' ,'PTC' ,'NVTL' ,'CNXC' ,'ONTO' ,'TTMI' ,'MXL' ,'PAYA' ,'SPSC' ,'FN' ,'ASX' ,'ROG' ,'AOSL' ,'GSCN' ,'VERI' ,'PUBM' ,'DOMO' ,'DCT' ,'FARO' ,'CNDT' ,'SMTC' ,'DCBO' ,'EXLS' ,'UIS' ,'GIB' ,'SUMO' ,'CSGS' ,'TLS' ,'HSTX' ,'BCOV' ,'SLTD' ,'CGNT' ,'LLNW' ,'VSCI' ,'ICHR' ,'NVMM' ,'TTEC' ,'VICR' ,'SMMC' ,'TSEMF' ,'ATMR' ,'PRFT' ,'MAXNV' ,'PLXS' ,'OSIS' ,'AUDC' ,'AKTS' ,'ZUO' ,'CSOD' ,'SANM' ,'VCRA' ,'NPTN' ,'CMTL' ,'EVTC' ,'VECO' ,'DMRC' ,'ALGM' ,'SEMR' ,'SITM' ,'JAMF' ,'CMBM' ,'BHE' ,'DNJR' ,'KN' ,'SATS' ,'IMMR' ,'EVOP' ,'EPAY' ,'YEXT' ,'MITK' ,'SVMK' ,'CRCT' ,'TBRG' ,'ESE' ,'ACLS' ,'WIT' ,'SPU' ,'XGTI' ,'ACVA' ,'PRGS' ,'MANT' ,'DBD' ,'NTCT' ,'INTZ' ,'UPLD' ,'DV' ,'ACMR' ,'MRIN' ,'BEEM' ,'CASA' ,'PLAB' ,'TSRA' ,'ECOM' ,'MICT' ,'MODN' ,'BOXL' ,'FTCI' ,'MLCH' ,'GGNT' ,'CANNY' ,'DSP' ,'NVMI' ,'AAOI' ,'GBOX' ,'VDSI' ,'MSP' ,'LCA' ,'PIOS' ,'ATEN' ,'WRAP' ,'PCVA' ,'AVGOP' ,'ZDGE' ,'METHA' ,'HLIT' ,'CAMT' ,'PRO' ,'LPL' ,'RDWR' ,'OLO' ,'SMCI' ,'SEAC' ,'AMTC' ,'COMP' ,'CTS' ,'DOCN' ,'ATC']
May_4 = [BasicMaterials, ConsumerCyclical, FinancialServices, RealEstate, ConsumerDefensive, Healthcare,\
           Utilities, CommunicationServices, Energy, Industrials, Technology]

In [8]:
# June 1 filtered_stocks (dollar_volume_min_price)
BasicMaterials = ['RIO' ,'FCX' ,'X' ,'ABX' ,'NEM' ,'NUE' ,'CLF' ,'DOW' ,'SHW' ,'PX' ,'ECL' ,'POT' ,'AA' ,'DOW' ,'AEM' ,'MLM' ,'PPG' ,'LYB' ,'APD' ,'CTVA' ,'RTP' ,'MOS' ,'IFF' ,'CE' ,'PCU' ,'AXTA' ,'STLD' ,'ALB' ,'VMC' ,'IST' ,'TCK' ,'BBL' ,'AVTR' ,'SMG' ,'KGC' ,'BHP' ,'GGB' ,'HL' ,'HUN' ,'SLW' ,'CF' ,'EMN' ,'FMC' ,'AU' ,'DRFNY' ,'KL' ,'GEVO' ,'FVAC' ,'SQM' ,'PAASF' ,'RGLD' ,'UFS' ,'AG' ,'ASH' ,'AUY' ,'FNV' ,'CX' ,'AMRS' ,'LAC' ,'OLN' ,'SSRIF' ,'CDE' ,'GRA' ,'HGMCY' ,'LTHM' ,'SUM' ,'LOAK' ,'FULL' ,'CXP' ,'CC' ,'CENX' ,'SID' ,'BTG' ,'WFG' ,'RPOW' ,'EXK' ,'RS' ,'WLK' ,'ESI' ,'BCC' ,'PLLL' ,'SWM' ,'BVN' ,'UFPI' ,'CRHCY' ,'SBSW' ,'KWR' ,'FSM' ,'CBT' ,'AGI' ,'GPRE' ,'KALU' ,'NGVT' ,'ABH' ,'CMC' ,'SAND' ,'GSM' ,'MDU' ,'UFC' ,'FPGP' ,'TROX' ,'PKX' ,'TSE' ,'EGO' ,'UNVR' ,'ZEUS' ,'WDFC' ,'HBM' ,'TMST' ,'SCHN' ,'PVG' ,'EQX' ,'POL' ,'EY' ,'MEOHF' ,'MTX' ,'SILV' ,'HCC' ,'CSTM' ,'GATO' ,'MVG' ,'BCP' ,'ZY' ,'KRA' ,'SSLT' ,'USCR' ,'USGL' ,'SA' ,'GCP' ,'IAG' ,'DRJ' ,'WRN' ,'HUGO' ,'CMP' ,'OR' ,'SVM' ,'MWGP' ,'SUZ' ,'GLT' ,'PQG' ,'EVA' ,'FF' ,'CTRA' ,'VRS' ,'OTL' ,'MERCS' ,'IPI' ,'SXC' ,'SCL' ,'USAS' ,'NG' ,'GSR' ,'CLW' ,'AXU' ,'ASIX' ,'DROOY']
ConsumerCyclical = ['TSLA' ,'AMZN' ,'GME' ,'F' ,'BABA' ,'HD' ,'PTON' ,'GM' ,'DKNG' ,'CCL' ,'ULTA' ,'MCD' ,'LOW' ,'NCLH' ,'JD' ,'EBAY' ,'NKE' ,'FTCH' ,'BBY' ,'PCLN' ,'GPS' ,'TJX' ,'SBUX' ,'KCAC' ,'MELI' ,'BBBY' ,'LVS' ,'W' ,'LULU' ,'EXPE' ,'CMG' ,'ETSY' ,'MGG' ,'RCL' ,'ORLY' ,'VTIQ' ,'AZO' ,'CVNA' ,'ROST' ,'BLL' ,'PENN' ,'MAR' ,'WSGC' ,'LTD' ,'KSS' ,'ERI' ,'DLPH' ,'FD' ,'YUMC' ,'CROX' ,'BURL' ,'CTRP' ,'DPZ' ,'WYNN' ,'NAKD' ,'IP' ,'DKS' ,'DHI' ,'YUM' ,'AEOS' ,'EXPR' ,'GOED' ,'TSCO' ,'WISH' ,'VFC' ,'PHM' ,'OSTK' ,'WHR' ,'NOBE' ,'GRWG' ,'WOOF' ,'RH' ,'AAP' ,'CHWY' ,'POOL' ,'WKHS' ,'DRI' ,'LEN' ,'HLT' ,'SPAQ' ,'LMTR' ,'TXRH' ,'KORS' ,'FBHS' ,'CPNG' ,'COH' ,'CCK' ,'AMCR' ,'MGA' ,'PVH' ,'URBN' ,'SHAK' ,'HIBB' ,'MHK' ,'SEE' ,'YETI' ,'UA' ,'TRIP' ,'DECK' ,'RIDE' ,'FIVE' ,'SFIX' ,'ANF' ,'BFAM' ,'HTHT' ,'RL' ,'GPC' ,'LKQX' ,'Z' ,'ELY' ,'STAY' ,'VAC' ,'LEVI' ,'WTW' ,'RII' ,'BYD' ,'HOME' ,'NVR' ,'BG' ,'THO' ,'EAT' ,'TPX' ,'WH' ,'QSR' ,'WRK' ,'XSPL' ,'UARM' ,'GT' ,'FND' ,'ASO' ,'RMG' ,'RACE' ,'KMX' ,'BWA' ,'FCAU' ,'CBRL' ,'CHDN' ,'CWH' ,'TOL' ,'PII' ,'ARVL' ,'MPEL' ,'LEA' ,'RVLV' ,'CRI' ,'VRM' ,'GPK' ,'TRY' ,'TTE' ,'PKG' ,'BERY' ,'CAKE' ,'HBI' ,'HRB' ,'GNTX' ,'SHLL' ,'BLNK' ,'XL' ,'SIX' ,'GES' ,'MTN' ,'REAL' ,'FOXF' ,'ARMK' ,'CHH' ,'SRV' ,'BC' ,'ALV' ,'VSTO' ,'LGIH' ,'MDC' ,'HDI' ,'DSW' ,'GCA' ,'AIRB' ,'PLCE' ,'BLMN' ,'POSH' ,'ROL' ,'SERV' ,'SIGYY' ,'ADNT' ,'GPI' ,'HAS' ,'PLNT' ,'FLWS' ,'TRWH' ,'KBH' ,'COLM' ,'KAR' ,'ALSN' ,'MTH' ,'HZO' ,'MAT' ,'SKX' ,'PLAY' ,'TEN' ,'CTB' ,'WGO' ,'IGT' ,'MED' ,'LOVE' ,'BZUN' ,'FM' ,'VC' ,'DDS' ,'QRTEA' ,'CUK' ,'GOOS' ,'BGFV' ,'WING' ,'HGV' ,'HMC' ,'RRR' ,'MLHR' ,'SHOO' ,'CCS' ,'MUSA' ,'KTB' ,'FNKO' ,'TOYOY' ,'BBW' ,'GOEV' ,'PZZA' ,'TMHC' ,'MMYT' ,'SON' ,'ABG' ,'TTM' ,'KIRK' ,'FUN' ,'HNST' ,'ACAM' ,'FUV' ,'CNNE' ,'GIL' ,'LEG' ,'SOLO' ,'DAN' ,'BOOT' ,'RBIN' ,'GLBE' ,'SKY' ,'FRSX' ,'GOLF' ,'MHO' ,'FOSL' ,'CTRN' ,'MBUU' ,'TKAT' ,'FTDRV' ,'SBH' ,'XPEL' ,'AMWD' ,'MRA' ,'ATR' ,'TPH' ,'DMYT' ,'H' ,'PRTY' ,'SEAS' ,'BKE' ,'LESL' ,'LCII' ,'GPAC' ,'HOV' ,'IHOP' ,'UAG' ,'CHCS' ,'WWW' ,'EYE' ,'SLGN' ,'OI' ,'LL' ,'REYN' ,'PRTS' ,'FHRI' ,'GIII' ,'BIOF' ,'MODI' ,'MNRO' ,'INSU' ,'RRGB' ,'LEDMV' ,'AIN' ,'GAN' ,'TDUP' ,'ODP' ,'LZB' ,'CVCO' ,'OMAD' ,'QRTEP' ,'GTX' ,'LCA' ,'DFXI' ,'GCO' ,'AXL' ,'SPWH' ,'IFSIA' ,'TCS' ,'SAH' ,'DFH' ,'HAVT' ,'CHGO' ,'CONN' ,'ZUMZ' ,'TUP' ,'ARCO' ,'CNGR' ,'ETH' ,'DRVN' ,'RNTA' ,'EZGO' ,'PATK' ,'AUVI' ,'DELT' ,'GDEN' ,'WPRT' ,'ARGNA' ,'VRA' ,'DOOO' ,'BZH' ,'SCVL' ,'RCKY' ,'PLOW' ,'CBD' ,'DINE' ,'CHUY' ,'MCFT' ,'RICK' ,'GBCOA' ,'NCT' ,'EFOI' ,'MOVA' ,'AYRO' ,'GMBL' ,'CLAC' ,'INVZ' ,'ONEW' ,'LQDT' ,'GRIL' ,'KNDI' ,'CARS' ,'VNE' ,'BNED' ,'PLYA' ,'RUTH' ,'RUSHA' ,'IPW' ,'JOAN' ,'JILL' ,'OXM' ,'DESP']
FinancialServices = ['V' ,'PYPL' ,'MA' ,'GS' ,'MWD' ,'BRKB' ,'TRV' ,'CMB' ,'NB' ,'BRKA' ,'NOB' ,'MHP' ,'COF' ,'AXP' ,'UPST' ,'USB' ,'TFC' ,'SCH' ,'BLK' ,'AOC' ,'ICE' ,'ACL' ,'MARA' ,'RKT' ,'WLTW' ,'ALL' ,'LMND' ,'MET' ,'CME' ,'HBAN' ,'DNB' ,'RY' ,'PRU' ,'PGR' ,'MMC' ,'SYF' ,'TROW' ,'RGBK' ,'CFG' ,'AFL' ,'BK' ,'BX' ,'TD' ,'ALLY' ,'FITB' ,'STT' ,'ITUB' ,'AIG' ,'HIG' ,'SPC' ,'MKTX' ,'CSR' ,'SIVB' ,'BAP' ,'AJG' ,'CCIV' ,'AMP' ,'EQH' ,'PNC' ,'DFS' ,'KKR' ,'TW' ,'RJF' ,'BBD' ,'KEY' ,'RE' ,'BMO' ,'EBC' ,'AVZ' ,'FRC' ,'FES' ,'BNS' ,'PFG' ,'MXB' ,'NTRS' ,'BCM' ,'RNR' ,'SI' ,'HBC' ,'HDB' ,'ADS' ,'CBOE' ,'APO' ,'IBN' ,'NDAQ' ,'SBNY' ,'IBKR' ,'UNM' ,'LNC' ,'OMF' ,'CINF' ,'ORI' ,'TREE' ,'ROOT' ,'FTEN' ,'BCS' ,'PSTH' ,'DB' ,'MKL' ,'CMA' ,'ATH' ,'ZION' ,'ING' ,'QCSB' ,'WU' ,'FAF' ,'FNT' ,'VOYA' ,'PBCP' ,'SLM' ,'BEN' ,'XP' ,'GHIV' ,'UBS' ,'EWBC' ,'WBST' ,'CHFC' ,'MFC' ,'ARES' ,'VIRT' ,'AIZ' ,'LPLA' ,'RCHI' ,'AFG' ,'SEIC' ,'MTG' ,'EVR' ,'LUK' ,'LTR' ,'PSLV' ,'CBSH' ,'FHB' ,'CG' ,'Y' ,'ESPD' ,'BKLY' ,'PBCT' ,'STIC' ,'FRHC' ,'FDS' ,'TMK' ,'RGA' ,'ESNT' ,'BPOP' ,'WAL' ,'JHG' ,'AMG' ,'FCNCA' ,'SF' ,'RILY' ,'MCMC' ,'UNIT' ,'ARCC' ,'SNV' ,'NAVIV' ,'COWND' ,'PFSI' ,'TRUP' ,'BXS' ,'PBR' ,'CACC' ,'PRSP' ,'HLI' ,'PNFP' ,'HEC' ,'FCBP' ,'OZRK' ,'FNC' ,'ACTC' ,'EBC' ,'TCBI' ,'FBP' ,'CNO' ,'FSIC' ,'COLB' ,'WPF' ,'PRI' ,'SLC' ,'VLY' ,'PHYS' ,'BHFWV' ,'MGI' ,'CCX' ,'AFC' ,'CIT' ,'ERIE' ,'BTNB' ,'FBAN' ,'BBV' ,'FFC' ,'FRX' ,'MORN' ,'GSHD' ,'RLI' ,'AXS' ,'LPRO' ,'GDOT' ,'TSIA' ,'SRNG' ,'WTFC' ,'HLNE' ,'LAZ' ,'HTGC' ,'FOCS' ,'VRTS' ,'SLQT' ,'ASB' ,'PSEC' ,'TYP' ,'GOCO' ,'FHI' ,'CMT' ,'LOB' ,'BKU' ,'ABCB' ,'UMPQ' ,'LC' ,'ISBC' ,'APAM' ,'FLGS' ,'GBCI' ,'CADE' ,'BOH' ,'CFR' ,'SC' ,'INDB' ,'LYG' ,'GNW' ,'HBHC' ,'MUFG' ,'FFIN' ,'PACX' ,'FRME' ,'MOTV.U' ,'EHTH' ,'MUDS' ,'KNSL' ,'CASH' ,'SIGI' ,'SAN' ,'ORCC' ,'CATY' ,'GHVI' ,'AVK' ,'IPOF' ,'SMFG' ,'SFTW' ,'PLMR' ,'UMBF' ,'UBSI' ,'GOF' ,'DCRN' ,'GOAC' ,'CBU' ,'LCY' ,'GGAL' ,'THBR' ,'SEAH' ,'BOKF' ,'NARA' ,'PDI' ,'SSPK' ,'IBTX' ,'PAWN' ,'LFCO' ,'CUBI' ,'NFJ' ,'CLM' ,'BOFI' ,'ATCO' ,'GCMG' ,'AACQ' ,'PCI' ,'MCY' ,'PUK' ,'FMBI' ,'NRK' ,'CAPA' ,'BMA' ,'CEF' ,'FSKR' ,'ENVA' ,'FBF' ,'AEL' ,'NMIH' ,'TBBKD' ,'HCII' ,'IBOC' ,'EFC' ,'JRS' ,'EIS' ,'EGBN' ,'WFSL' ,'FTOC' ,'PDO' ,'OLDB' ,'OXLC' ,'VGAC' ,'AC' ,'VACQ' ,'TBA' ,'WD' ,'FAII' ,'GSBD' ,'UCBI' ,'RBS' ,'FWWB' ,'FULT' ,'AEG' ,'WSBC' ,'KB' ,'ARC' ,'BHL' ,'DFIN' ,'MC' ,'STD' ,'HPX' ,'AGO' ,'SFNCA' ,'BMEZ' ,'MAIN' ,'AMRK' ,'NSTB' ,'SFBS' ,'HMN' ,'PFS' ,'APGB' ,'PTY' ,'PJT' ,'GBDC' ,'BSTZ' ,'CFFN' ,'HOMB' ,'DNP' ,'CNS' ,'BANF' ,'ESGRD' ,'FIBK' ,'CNA' ,'RTP' ,'WSFS' ,'BDJ' ,'NAD' ,'STEP' ,'JRVR' ,'PTA' ,'PHC' ,'ANAT' ,'FDEF' ,'BSBR' ,'BTWN' ,'JWS' ,'PAX' ,'SHG' ,'ASZ' ,'OHSC' ,'CLII' ,'TSLX' ,'RA' ,'NCZ' ,'PRAA' ,'EIG' ,'GAB' ,'PIPR' ,'INSP' ,'OCFC' ,'FIII' ,'BCAT' ,'THCB' ,'FPTB' ,'WASH' ,'SASR' ,'DSL' ,'GSVC' ,'FPF' ,'TPGY' ,'EXG' ,'CRVL' ,'AGC' ,'LFC' ,'FSRV' ,'FFBC' ,'FAMCK' ,'INDI' ,'NVG' ,'BRP' ,'RQI' ,'BSPE' ,'OMAM' ,'UBSH' ,'CCV' ,'TRMK' ,'NEA' ,'FORE' ,'NCV' ,'GWB' ,'FSC' ,'PXT' ,'HQH' ,'NTB' ,'PSAC' ,'USA' ,'OFG' ,'TRST' ,'CSWC' ,'TFSL' ,'CMLF' ,'STC' ,'CVB' ,'FVH' ,'APXT' ,'UTG' ,'CPUH' ,'UTF' ,'TBK' ,'GDV' ,'NWSB' ,'ETY' ,'EBSB' ,'CIB' ,'GSAH' ,'CHCO' ,'BOWX' ,'AMBC' ,'SRAC' ,'RM']
RealEstate = ['AMT' ,'AMB' ,'TWRS' ,'PSA' ,'SPG' ,'O' ,'EQIXD' ,'SBAC' ,'BYA' ,'AHT' ,'WY' ,'EQR' ,'ARE' ,'RIGX' ,'NLY' ,'OPEN' ,'CBRE' ,'HCN' ,'DLR' ,'EXR' ,'ARCP' ,'VICI' ,'DRE' ,'CDC' ,'RLGY' ,'AGNC' ,'ESS' ,'VC' ,'AMH' ,'MPW' ,'BXP' ,'INVH' ,'MAA' ,'MHC' ,'CPT' ,'LAP' ,'KIM' ,'IMTN' ,'SVC' ,'REXR' ,'MAC' ,'YSI' ,'OHI' ,'COLD' ,'RDFN' ,'UDR' ,'CUZ' ,'PEAK' ,'SRC' ,'SUI' ,'REG' ,'STWD' ,'VNO' ,'LSI' ,'ACC' ,'PK' ,'CONE' ,'DOC' ,'SLG' ,'HTA' ,'NRZ' ,'CBSO' ,'SUNH' ,'ADC' ,'STOR' ,'SKT' ,'IVR' ,'FRT' ,'FR' ,'NSA' ,'KRC' ,'NTAIF' ,'BNL' ,'HIW' ,'GLPIV' ,'LAMR' ,'STIR' ,'COR' ,'EPR' ,'BPY' ,'NHI' ,'BRX' ,'EXPI' ,'DEI' ,'CT' ,'RYN' ,'APLE' ,'EGP' ,'NNN' ,'SHO' ,'PGRE' ,'COPT' ,'LTC' ,'WPG' ,'IIPR' ,'ABR' ,'HR' ,'ESRT' ,'HRP' ,'CLNY' ,'WHC' ,'BDN' ,'DDR' ,'HASI' ,'CXP' ,'HPP' ,'GET' ,'WRI' ,'JBGS' ,'TRNO' ,'FCPT' ,'LXP' ,'CSALV' ,'AIRC' ,'PEB' ,'QTS' ,'RLJ' ,'PCH' ,'TWO' ,'CIM' ,'HHC' ,'MGP' ,'PSM' ,'AIV' ,'DRH' ,'NAQ' ,'WRE' ,'KREF' ,'INN' ,'CTREV' ,'NTST' ,'EPRT' ,'AAT' ,'PDM' ,'EST' ,'PAX' ,'GOOD' ,'MNRTA' ,'PMT' ,'IRT' ,'LAND' ,'GNL' ,'ARI' ,'PZN' ,'ORC' ,'RRF' ,'RWTI' ,'CLI' ,'NMRK' ,'AFIN' ,'CWK' ,'UE' ,'XHR' ,'RPAI' ,'APT' ,'NTR' ,'AHH' ,'MCT' ,'MMI' ,'ZFC' ,'SRG' ,'BPR' ,'GPMT' ,'ALEX' ,'MFA' ,'SJP' ,'BRG' ,'DEA' ,'KRG' ,'SAFE' ,'ILPT' ,'SNH' ,'GOV' ,'SNR' ,'NXRT' ,'MITT' ,'HT' ,'GMRE' ,'AHP' ,'DX' ,'MDJH' ,'UMH' ,'BRMK' ,'LADR']
ConsumerDefensive = ['BYND' ,'COST' ,'PG' ,'WMT' ,'KO' ,'PEP' ,'DG' ,'DLTR' ,'KFT' ,'DH' ,'KMB' ,'SAM' ,'CL' ,'PM' ,'EL' ,'MO' ,'OLLI' ,'KR' ,'CBRNA' ,'EDU' ,'CNS' ,'GIS' ,'SJMA' ,'K' ,'KHC' ,'ADM' ,'DPS' ,'CLX' ,'HANS' ,'MCCRK' ,'HSY' ,'SYY' ,'BTI' ,'BUD' ,'ACCOB' ,'TSN' ,'CPB' ,'CAG' ,'BG' ,'DAR' ,'NWL' ,'LW' ,'UL' ,'HRL' ,'ABEV' ,'DEO' ,'CELH' ,'CHGG' ,'PFGC' ,'USFD' ,'CHD' ,'COUR' ,'FRPT' ,'CASY' ,'SFM' ,'FMX' ,'BJ' ,'SAFM' ,'CCEP' ,'HLF' ,'COTY' ,'HELE' ,'FLO' ,'BFB' ,'VFF' ,'FIZ' ,'PDA' ,'TPB' ,'BGS' ,'NOMD' ,'OAC' ,'UTZ' ,'BGL' ,'CPO' ,'TWOU' ,'CORE' ,'ARCE' ,'THS' ,'GRSH' ,'NOVS' ,'UVV' ,'LANC' ,'UNFI' ,'GO' ,'AGRO' ,'ACI' ,'LOPE' ,'COKE' ,'SPB' ,'HAIN' ,'SMPL' ,'FMCI' ,'POST' ,'PRMW' ,'STRA' ,'ELF' ,'NUS' ,'EPC' ,'DV' ,'LAUR' ,'LRN' ,'CENTA' ,'JJSF' ,'VITL' ,'WPO' ,'BRBR' ,'HMHC' ,'PPC' ,'CVGW' ,'CALM' ,'CECO' ,'KOF' ,'CHEF' ,'SPTN' ,'TR' ,'FAMI' ,'AFYA' ,'NTCO' ,'BFA' ,'USNA' ,'CENT' ,'EEIQ' ,'FREE' ,'STKLF' ,'IMKTA' ,'WMK' ,'JEAN' ,'PSMT' ,'METX' ,'AQBTV']
Healthcare = ['MRNA' ,'JNJ' ,'UNH' ,'TLRY' ,'AMGN' ,'VEEV' ,'TMO' ,'SNDL' ,'CVS' ,'MDT' ,'ABBV' ,'ABT' ,'DHR' ,'ISRG' ,'PFE' ,'ATH' ,'GILD' ,'TDOC' ,'MRK' ,'LLY' ,'VRTX' ,'BDX' ,'ILMN' ,'IQV' ,'CGC' ,'HSGX' ,'DXCM' ,'BAX' ,'ZEN' ,'REGN' ,'BMY' ,'CNTE' ,'CI' ,'IDPH' ,'ZMH' ,'NOX' ,'ZTS' ,'BSX' ,'HUM' ,'MCK' ,'ALGN' ,'HCA' ,'CRL' ,'WAG' ,'EW' ,'ALXN' ,'SYK' ,'STAA' ,'CERN' ,'OSH' ,'ACB' ,'IDXX' ,'A' ,'HZNP' ,'EXAS' ,'WST' ,'SGEN' ,'GLX' ,'UHS' ,'RCM' ,'CRON' ,'LH' ,'TECH' ,'PODD' ,'NVS' ,'NVCR' ,'TXG' ,'BCRX' ,'STRL' ,'CRSP' ,'IPOC' ,'DGX' ,'HOLX' ,'EBS' ,'AAS' ,'RESM' ,'CAH' ,'ZLAB' ,'ICLRY' ,'ELAN' ,'GDRX' ,'INCY' ,'WAT' ,'NTLA' ,'HEXO' ,'ABMD' ,'NTRA' ,'ITRM' ,'AVII' ,'BHVN' ,'EYES' ,'XRAY' ,'PEN' ,'PRAH' ,'RETA' ,'BNGO' ,'HSIC' ,'HQY' ,'BGNE' ,'ALC' ,'MRTX' ,'MTD' ,'EGG' ,'COO' ,'AUPH' ,'PGNY' ,'PTIE' ,'NVTA' ,'BEAM' ,'MASI' ,'NVO' ,'GH' ,'ONVO' ,'NBIX' ,'TRL' ,'THC' ,'PRVB' ,'ARWR' ,'JAZZ' ,'TWST' ,'TFX' ,'PDCO' ,'ALLO' ,'IOVA' ,'OMI' ,'ISIP' ,'SWAV' ,'NVST' ,'SAGE' ,'HB' ,'CTLT' ,'ONEM' ,'VTRSV' ,'BPMC' ,'BIOA' ,'FLGT' ,'HGEN' ,'OGI' ,'KBIO' ,'TEVIY' ,'ALNY' ,'NNOX' ,'QDEL' ,'BVF' ,'EXEL' ,'PPD' ,'CVAC' ,'PACB' ,'SDGR' ,'SRNE' ,'FATE' ,'CDNA' ,'ARGX' ,'MOH' ,'UTHR' ,'CHNG' ,'BMRN' ,'HAE' ,'VIR' ,'INCR' ,'NARI' ,'GEB' ,'RGEN' ,'TGTX' ,'PHG' ,'CNTL' ,'TNDM' ,'ACCD' ,'INMD' ,'RPRX' ,'PRGO' ,'AHPA' ,'ARVN' ,'GTHX' ,'ABCL' ,'HRC' ,'AMWL' ,'INSP' ,'LIVN' ,'HTI' ,'SDC' ,'AMED' ,'PVTR' ,'MNMD' ,'GMED' ,'SYRS' ,'CNAC' ,'NVRO' ,'HRTX' ,'AXNX' ,'NSTG' ,'SENS' ,'PINC' ,'OMCL' ,'SNY' ,'LNTH' ,'PDX' ,'DNLI' ,'BLI' ,'IRTC' ,'BLUE' ,'MRVI' ,'LHCG' ,'RAD' ,'TPTX' ,'PCRX' ,'KRTX' ,'NABI' ,'ANVS' ,'BDAL' ,'PIRS' ,'GBT' ,'CHE' ,'EDIT' ,'RARE' ,'ARNA' ,'ALKS' ,'APLS' ,'AHS' ,'ADPT' ,'FOLD' ,'JYNT' ,'BBIO' ,'EAR' ,'INOV' ,'QGENF' ,'DRNA' ,'XLRN' ,'ZOM' ,'SCYX' ,'EDGE' ,'CCXX' ,'SGRY' ,'RVMD' ,'MNKD' ,'OCDX' ,'ACAD' ,'TTOO' ,'ICUI' ,'INSM' ,'MGLN' ,'NGM' ,'SIBN' ,'GMAB' ,'HCAT' ,'XXII' ,'GRFS' ,'PRLD' ,'PHR' ,'ATOS' ,'CLVR' ,'SNN' ,'SEEL' ,'ENDP' ,'LGND' ,'ICPT' ,'ZNTL' ,'AGEN' ,'PETQ' ,'ZYNE' ,'NUVA' ,'SILK' ,'GKOS' ,'CVETV' ,'MMSI' ,'GLPG' ,'SEM' ,'QTRX' ,'PETS' ,'IMGN' ,'PTCT' ,'MORF' ,'CRBP' ,'JAGX' ,'AXSM' ,'AHCO' ,'SEER' ,'QSII' ,'STOK' ,'MDRX' ,'ETON' ,'HIV' ,'OM' ,'KOD' ,'EBIO' ,'BDTX' ,'ALLK' ,'MDGL' ,'MEDP' ,'MYOV' ,'CERT' ,'AGIO' ,'CYH' ,'TYHT' ,'TMDI' ,'NEOT' ,'CNMD' ,'LCI' ,'ENSG' ,'SHC' ,'VSTM' ,'TVTX' ,'CLVS' ,'KALA' ,'TAK' ,'ASTM' ,'NEO' ,'FGEN' ,'ARCT' ,'VCYT' ,'CRIS' ,'IART' ,'PSNL' ,'ACHC' ,'CYTK' ,'MCRB' ,'CCXI' ,'SWTX' ,'TBIO' ,'ITCI' ,'GLSI' ,'KYMR' ,'PTGX' ,'KURA' ,'GBIO' ,'PRAX' ,'NRIX' ,'AFIB' ,'LMNX' ,'LUNG' ,'RXII' ,'RGNX' ,'SELB' ,'ATEC' ,'CTXR' ,'NEOT' ,'NEOG' ,'SRRK' ,'CCCC' ,'DVAX' ,'THRX' ,'BTAI' ,'TNXP' ,'KBLM' ,'PRVA' ,'CDXS' ,'RTIX' ,'IRWD' ,'NBRV' ,'RLAY' ,'ALDX' ,'DCPH' ,'SOLY' ,'TPIV' ,'PBH' ,'ALHC' ,'CGEN' ,'ATRC' ,'ZYME' ,'NOVN' ,'AVXL' ,'MGNX' ,'AVIR' ,'HSKA' ,'HYH' ,'OCUL' ,'ATHA' ,'CORT' ,'OPCH' ,'ACRS' ,'SGMO' ,'QURE' ,'EVH' ,'ATRA' ,'CHRS' ,'CERS' ,'CTMX' ,'XERS' ,'CHNGU' ,'SNDX' ,'KITS' ,'TCLN' ,'RCUS' ,'RDHL' ,'RUBY' ,'AFMD' ,'SURF' ,'PMVP' ,'PRPO' ,'HRMY' ,'INHL' ,'ADUS' ,'RVNC' ,'FHC' ,'HCSG' ,'CSTL' ,'IGMS' ,'REPL' ,'CYPH' ,'VNDA' ,'TRHC' ,'MASS' ,'ALXO' ,'ALT' ,'TSHA' ,'OSCR' ,'KPTI' ,'TYME' ,'PAVM' ,'MTEC' ,'ESPR' ,'PRTA' ,'XNCR' ,'AGTI' ,'ONTX' ,'OMER' ,'AGL' ,'EPTO' ,'YMAB' ,'MYGN' ,'RXRX' ,'CRTX' ,'FMTX' ,'AKBA' ,'SANA' ,'PRQR' ,'ITEK' ,'IBIO' ,'VAPO' ,'BKD' ,'ANGO' ,'EARS' ,'SUPN' ,'MBIO' ,'FMS' ,'OLMA' ,'CATS' ,'RDYN' ,'CODX' ,'AERI' ,'ARDX' ,'ATRI' ,'KRON' ,'HSAC' ,'BIOD' ,'AKRO' ,'ATNX' ,'PNTG' ,'TCMD' ,'RHE' ,'BLFS' ,'HCCO' ,'ZGNX' ,'EOLS' ,'DPII' ,'TCEL' ,'ALEC' ,'CUTR' ,'PASG' ,'MRUS' ,'ANAB' ,'WVE' ,'PRSC' ,'VRAY' ,'MRSN' ,'KNSA' ,'XON' ,'LIFE' ,'RIGL' ,'ADCT' ,'CPRX' ,'AMTI' ,'CNDO' ,'AVAH' ,'RDY' ,'PLCC' ,'SCPS' ,'COLL' ,'VKTX' ,'ABCM' ,'ALVR' ,'OPRX' ,'DRIO' ,'ESTA' ,'KROS' ,'ELOX' ,'STTK' ,'EGRX' ,'ZIOP' ,'TXMD' ,'ARQT' ,'CGEM' ,'VTGN' ,'MRIC' ,'INGN' ,'AAVL' ,'CNST' ,'CMRX' ,'MNLO' ,'CBYL' ,'ANTR' ,'RYTM' ,'GTBP' ,'CLLS' ,'GB' ,'ENTA' ,'TMCI' ,'FLDM' ,'RVP' ,'HCM' ,'RDUS' ,'MDXG' ,'ACST' ,'ANIK' ,'KIDS' ,'SGFY' ,'KTOV' ,'SIMU' ,'ATNM' ,'DTIL' ,'CARA' ,'BDXB' ,'ODT' ,'TIL' ,'TMDX' ,'MNOV' ,'STRO' ,'RNA' ,'LMAT' ,'GOSS']
Utilities = ['FPL' ,'SRE' ,'D' ,'PE' ,'DUK' ,'AEP' ,'NSP' ,'SO' ,'PCG' ,'ED' ,'AEE' ,'CMS' ,'NU' ,'NRG' ,'PEG' ,'HOU' ,'PPL' ,'DTE' ,'WEC' ,'NI' ,'VST' ,'WPH' ,'AWK' ,'AES' ,'OGE' ,'FE' ,'WR' ,'ETR' ,'UGI' ,'EIX' ,'ATO' ,'PNW' ,'PNM' ,'NGG' ,'ORA' ,'WTRG' ,'IDA' ,'NEP' ,'LG' ,'PCGU' ,'AGR' ,'BEPC' ,'BEPCW' ,'OGS' ,'FTS' ,'CWEN' ,'HE' ,'NJR' ,'BIPC' ,'SWX' ,'SJI' ,'POR' ,'WWP' ,'BKH' ,'ABY' ,'NWEC' ,'NFE' ,'SBS' ,'SCW' ,'CPK' ,'CIG' ,'MPL' ,'OEG' ,'NWNG' ,'BIP' ,'AQN' ,'AZRE' ,'ENI' ,'BEP' ,'MSEX' ,'ELP' ,'EBR' ,'CWT' ,'CWEN.A' ,'UTL' ,'CLCI' ,'DCUE' ,'OTTR' ,'MDSN' ,'DTP']
CommunicationServices = ['FB' ,'AMC' ,'GOOG' ,'GOOCV' ,'ABNB' ,'NFLX' ,'RBLX' ,'DIS' ,'SBC' ,'ZM' ,'BEL' ,'ROKU' ,'CMCSA' ,'SNAP' ,'VIAC' ,'BIDU' ,'PINS' ,'PCS' ,'TWTR' ,'Z' ,'TWLO' ,'FUBO' ,'DASH' ,'CHTR' ,'BILI' ,'DISCA' ,'ATVI' ,'MTCH' ,'SPOT' ,'FEAC' ,'IAC' ,'OMC' ,'NTES' ,'FVRR' ,'ERTS' ,'DISCK' ,'IPG' ,'TTWO' ,'CNK' ,'CDRD' ,'NWSAV' ,'CTL' ,'DISH' ,'ZNGA' ,'ATUS' ,'LBDKV' ,'YNDX' ,'LYV' ,'FOXAV' ,'CABO' ,'MOXC' ,'LSXMK' ,'ATHM' ,'LSXMA' ,'GRUB' ,'IDC' ,'RUBI' ,'ZG' ,'WWFE' ,'MSGS' ,'VOD' ,'BCE' ,'NYT' ,'FOXBV' ,'NGMS' ,'LBTYK' ,'LBTYA' ,'NXST' ,'TV' ,'AMX' ,'GHQ' ,'GCI' ,'LGF.A' ,'EDR' ,'VG' ,'AMCX' ,'MBT' ,'CDLX' ,'MSGE' ,'CARG' ,'MSGEW' ,'YELP' ,'LMCKV' ,'IMAXF' ,'GENI' ,'GOGO' ,'SBGI' ,'SSTK' ,'TEF' ,'GNUS' ,'RUK' ,'WIFI' ,'TU' ,'TDS' ,'MAX' ,'VIV' ,'SKM' ,'LGF.B' ,'GCSB' ,'IHRT' ,'AIX' ,'GRPN' ,'NWSVV' ,'COI' ,'LBDAV' ,'COUP' ,'KTC' ,'GCI' ,'ROIA' ,'ORBC' ,'RG' ,'MDP' ,'WMG' ,'VPRT' ,'AVID' ,'THRY' ,'SCR' ,'PLTK' ,'SHET' ,'SJR' ,'ROIAK' ,'MCS' ,'JWA' ,'SCPL' ,'RADI' ,'DMD' ,'LILKV' ,'HOFV' ,'TIMB' ,'WOW' ,'IZEA' ,'VIP' ,'SSP' ,'CNSL' ,'TTGT' ,'TRUE' ,'QNST' ,'DLX' ,'EVER' ,'CCO' ,'SAQN' ,'LIVX' ,'USM' ,'TLK' ,'SLGG' ,'OOMA' ,'NCMI' ,'WPP' ,'MAIL' ,'FTE']
Energy = ['VTNR' ,'XON' ,'CHV' ,'MPC' ,'PBR' ,'P' ,'BP' ,'OXY' ,'SLB' ,'HOC' ,'PXD' ,'KMI' ,'IPPIF' ,'VLO' ,'MRO' ,'EOG' ,'WMB' ,'RDSA' ,'OKE' ,'DVN' ,'MPET' ,'BHGE' ,'PSX' ,'ETE' ,'PBRA' ,'COG' ,'AHC' ,'FANG' ,'SU' ,'HAL' ,'EPD' ,'CCJ' ,'RDSB' ,'APA' ,'XEC' ,'TPL' ,'CHEX' ,'TOT' ,'SLCA' ,'STO' ,'AMGP' ,'CED' ,'EQT' ,'TRP' ,'CVE' ,'MPLX' ,'LOM' ,'MARY' ,'SWN' ,'RIG' ,'TRGP' ,'AR' ,'UUUU' ,'NOI' ,'WEG' ,'CLNE' ,'APY' ,'TPGE' ,'BTU' ,'SHLX' ,'DEN' ,'REGI' ,'ETRN' ,'OVV' ,'CHK' ,'CLNP' ,'NBR' ,'MUR' ,'PAGP' ,'VVV' ,'SRAQ' ,'CLR' ,'TS' ,'PAA' ,'DNN' ,'PETD' ,'DPM' ,'WGP' ,'CRLBF' ,'PSXP' ,'FTI' ,'DHT' ,'CVI' ,'PBF' ,'INT' ,'MTDR' ,'OMP' ,'HESM' ,'HP' ,'NRGY' ,'ARCH' ,'AMTX' ,'PBA' ,'CNX' ,'STNG' ,'IMO' ,'HEP' ,'ENLC' ,'WLL' ,'CDIS' ,'UEC' ,'LPI' ,'DK' ,'GLNG' ,'KOS' ,'XOG' ,'CRC' ,'BCEI' ,'CPG' ,'FRO' ,'PTEN' ,'CRK' ,'SUSP' ,'NFG' ,'VNOM' ,'ERF' ,'OII' ,'EXH' ,'UDL' ,'NOG' ,'EURN' ,'EC' ,'TRCH' ,'GEL' ,'LBRT' ,'SNP' ,'VET' ,'BPMP' ,'NXE' ,'CQP' ,'RCON' ,'PTR' ,'SGY' ,'OAS' ,'WHD' ,'WTI' ,'CEIX' ,'LPG' ,'PBFX' ,'MNRL' ,'YPF' ,'MCF' ,'ENBL' ,'URG' ,'REI' ,'DKL' ,'UGP' ,'GLOG' ,'E' ,'ERA' ,'DNOW']
Industrials = ['BA' ,'SPCE' ,'PLUG' ,'GE' ,'AAL' ,'CNI' ,'CAT' ,'DE' ,'WBT' ,'ALD' ,'UTX' ,'UAUA' ,'UNP' ,'UPS' ,'KSU' ,'TRU' ,'MIDD' ,'FDX' ,'GPN' ,'MMM' ,'NSC' ,'LMT' ,'AUD' ,'CUM' ,'DAL' ,'ETN' ,'LUV' ,'EMR' ,'PH' ,'ERC' ,'CP' ,'CSX' ,'NOC' ,'SWK' ,'HRS' ,'CARR' ,'CARRW' ,'ITW' ,'TDG' ,'VRSK' ,'ROK' ,'MRKT' ,'OTIS' ,'OTISW' ,'ROP' ,'EXPD' ,'GNRC' ,'AZEK' ,'WAB' ,'ODFL' ,'URI' ,'JCI' ,'GD' ,'EFX' ,'UW' ,'AVY' ,'CPRT' ,'CTAS' ,'RSG' ,'AME' ,'FAST' ,'AA' ,'JBLU' ,'PAYX' ,'PCAR' ,'CHRW' ,'SPR' ,'SZI' ,'LPX' ,'JBHT' ,'ALK' ,'MAS' ,'BLDPF' ,'IAA' ,'CD' ,'SAVE' ,'OC' ,'BLDR' ,'BLD' ,'IR' ,'VRT' ,'ZTO' ,'WCNX' ,'ALLE' ,'NAV' ,'SNA' ,'MTZ' ,'NLSN' ,'TXT' ,'DOV' ,'TASR' ,'RHI' ,'J' ,'HEI' ,'IEX' ,'XYL' ,'GWW' ,'WSO' ,'ZIM' ,'PWR' ,'BE' ,'AYI' ,'TGH' ,'NDSN' ,'ABB' ,'AOS' ,'HII' ,'ACM' ,'CNHI' ,'AER' ,'TWP' ,'PNR' ,'BAH' ,'DCI' ,'EAF' ,'KNGT' ,'VALM' ,'TOC' ,'AG' ,'ERJ' ,'UHAL' ,'TTC' ,'HYFM' ,'OTRKB' ,'UPWK' ,'HTZ' ,'WGOV' ,'CPA' ,'LII' ,'CFX' ,'SITE' ,'HA' ,'LSTR' ,'MESA' ,'JELD' ,'MAN' ,'GGG' ,'HXL' ,'RCII' ,'JBT' ,'WATR' ,'WMS' ,'RBC' ,'KRNT' ,'SBLK' ,'HEIA' ,'RBA' ,'PRLB' ,'TKR' ,'EMCG' ,'CW' ,'FLR' ,'GFL' ,'DY' ,'KBR' ,'CSL' ,'EAGL' ,'MMS' ,'ALT' ,'TEX' ,'TAL' ,'MRCY' ,'ENS' ,'HUBB' ,'WTS' ,'ACP' ,'AZUL' ,'R' ,'SNCY' ,'EOSE' ,'VRTV' ,'ULCC' ,'BWC' ,'KMT' ,'LECO' ,'MSM' ,'WERN' ,'AAN' ,'NVT' ,'PRG' ,'FTIC' ,'WCC' ,'LEV' ,'WFII' ,'B' ,'ANDA' ,'RYAAY' ,'IIN' ,'AWI' ,'TITN' ,'TPIC' ,'PAC' ,'SWBI' ,'VLRS' ,'AIMC' ,'ROLL' ,'SCST' ,'FIX' ,'MIC' ,'KODK' ,'APG' ,'CR' ,'ALGT' ,'IBP' ,'FLS' ,'GOL' ,'REZI' ,'FAIL' ,'ASR' ,'AAWW' ,'TRN' ,'TGI' ,'SRCL' ,'ATKR' ,'GSHT' ,'AL' ,'DAC' ,'ARNC' ,'ENR' ,'ASGN' ,'PZB' ,'ADT' ,'AQUA' ,'HI' ,'CATM' ,'AEIS' ,'GBX' ,'PRIM' ,'CYRX' ,'APZ' ,'CRS' ,'TNET' ,'RXN' ,'BLDG' ,'ENRJ' ,'ABFS' ,'DOOR' ,'CLHB' ,'SKYW' ,'ABM' ,'PSN' ,'UNF' ,'GVA' ,'SB' ,'GFF' ,'BECN' ,'TFII' ,'CFXA' ,'ERII' ,'OMAB' ,'ROCK' ,'AVAV' ,'ASF' ,'PBI' ,'KFY' ,'AIR' ,'SSD' ,'AQMS' ,'CMCO' ,'SFL' ,'ALEX' ,'MYRG' ,'ABXA' ,'HAYW' ,'GY' ,'CAP' ,'MWA' ,'BXC' ,'RAIL' ,'SWIM' ,'BWC' ,'AMRC' ,'HUBG' ,'KEX' ,'ACY' ,'VLCCF' ,'CWST' ,'RGR' ,'MLI' ,'XONE' ,'KNL' ,'ROAD' ,'AZZ' ,'TSP' ,'NMM' ,'SHYF' ,'ASTE' ,'BMI' ,'CGT' ,'WTHG' ,'POWW' ,'MTW' ,'FSS' ,'DPW' ,'GTES' ,'ACA' ,'MNES' ,'TANT' ,'AAON' ,'FELE' ,'FLOW' ,'HYRE' ,'LNN' ,'BW' ,'LAND' ,'TFT' ,'ABD' ,'SNDR' ,'SNHY' ,'KAMNA' ,'ECOL' ,'GMT' ,'DCO' ,'GFLU' ,'SPW' ,'FRTA' ,'VVNT' ,'SCS' ,'CCRN' ,'DGLY' ,'IVAC' ,'GRSHU' ,'RAVN' ,'NAT' ,'MOG.A' ,'SXI' ,'EPAC' ,'SMED' ,'GSTL' ,'CMRE' ,'BRCOA' ,'MRTN' ,'WNC' ,'HTLD' ,'ARCI' ,'WIRE' ,'EGLE' ,'GMS' ,'MAQ' ,'DNY' ,'PGTI' ,'MIII' ,'PFMT' ,'HONI' ,'SWT' ,'NPO' ,'MEG' ,'CBAK' ,'HEES' ,'DHC' ,'HURN' ,'ROMC' ,'HSC']
Technology = ['NVDA' ,'CRM' ,'MSFT' ,'AAPL' ,'AMD' ,'PLTR' ,'COIN' ,'SNOW' ,'INTC' ,'SHOP' ,'RIMM' ,'ADBE' ,'MU' ,'HWP' ,'NOW' ,'QCOM' ,'AVGO' ,'AMAT' ,'CSCO' ,'UBER' ,'SQ' ,'TSM' ,'OKTA' ,'TXN' ,'CRWD' ,'ACN' ,'INTU' ,'LYFT' ,'ADSK' ,'LRCX' ,'ORCL' ,'ADI' ,'LEDS' ,'NXPI' ,'ASMLF' ,'MVIS' ,'ZS' ,'WDAY' ,'CEY' ,'RNG' ,'DOCU' ,'WORK' ,'VSNR' ,'APPY' ,'TTD' ,'CTSH' ,'IBM' ,'NOK' ,'DELL' ,'VMW' ,'FISV' ,'KLAC' ,'TEAM' ,'ENPH' ,'PANW' ,'AHA' ,'RUN' ,'U' ,'MRVL' ,'XLNX' ,'SPLK' ,'STX' ,'TDSC' ,'AFRM' ,'MSTR' ,'ON' ,'BILL' ,'PLAN' ,'KOSS' ,'FSLY' ,'CTXS' ,'QRVO' ,'TER' ,'WDC' ,'SNPS' ,'CDW' ,'BMBL' ,'HPE' ,'PATH' ,'NET' ,'COUP' ,'MOT' ,'CDN' ,'GLW' ,'HUBS' ,'DDOG' ,'MCHP' ,'FTNT' ,'APH' ,'TEL' ,'MXIM' ,'WIX' ,'VRSN' ,'GART' ,'NTAP' ,'DBX' ,'CREE' ,'ZBRA' ,'MDB' ,'PFPT' ,'YEXT' ,'SYMC' ,'APPN' ,'FTV' ,'BR' ,'JFAX' ,'SEDG' ,'VUZI' ,'MNDL' ,'STNE' ,'KEYS' ,'BKI' ,'AI' ,'AKAM' ,'FLT' ,'WXS' ,'TDY' ,'TRMB' ,'EMA' ,'MOSY' ,'GRMN' ,'CHKPF' ,'ANSS' ,'FSLR' ,'AVLR' ,'BIGC' ,'BOX' ,'EPAM' ,'AVGOP' ,'PSTG' ,'GDS' ,'PAYC' ,'MPWR' ,'TYL' ,'ZEN' ,'CCC' ,'ANET' ,'CIEN' ,'AZPN' ,'SABR' ,'GDDY' ,'ESTC' ,'JKHY' ,'BSY' ,'IIVI' ,'XRX' ,'CDAY' ,'SPWR' ,'G' ,'PAGS' ,'ARRY' ,'GMHI' ,'LAZR' ,'LOGIY' ,'FIC' ,'ARW' ,'FIVN' ,'SAI' ,'CRCT' ,'ERICY' ,'NTNX' ,'DT' ,'FROG' ,'DOMO' ,'FAF' ,'DXC' ,'SSNC' ,'SAP' ,'FEYE' ,'BRKS' ,'IPGP' ,'JNPR' ,'FFIV' ,'NOVA' ,'PMTC' ,'EDA' ,'AYX' ,'CYBR' ,'MKSI' ,'INFY' ,'CGNX' ,'QTWO' ,'YOD' ,'FOUR' ,'STM' ,'LITEV' ,'CSIQ' ,'VLDR' ,'PUBM' ,'SMAR' ,'GWRE' ,'UNPH' ,'NCNO' ,'CRUS' ,'LSPD' ,'IRBT' ,'GPRO' ,'UMC' ,'COHR' ,'ST' ,'ENTG' ,'SONO' ,'JBIL' ,'CRSR' ,'ACXM' ,'TRNE' ,'SAIL' ,'SAIC' ,'FLEXF' ,'DLB' ,'AMBA' ,'GLOB' ,'MANH' ,'VNT' ,'VNTW' ,'PD' ,'COMM' ,'VSAT' ,'PEGA' ,'NCR' ,'DOX' ,'SWCH' ,'PRO' ,'CLDR' ,'OTEXF' ,'LSCC' ,'DCT' ,'EEFT' ,'KOPN' ,'MDLA' ,'ZI' ,'XM' ,'CVLT' ,'LPL' ,'ENV' ,'AVT' ,'CACI' ,'EVBG' ,'ASAN' ,'UCTT' ,'WRAP' ,'SSYS' ,'BAND' ,'CRNCV' ,'BL' ,'DNB' ,'SMTC' ,'PCTY' ,'RPD' ,'EB' ,'PANL' ,'KLIC' ,'RXT' ,'PRFT' ,'QLYS' ,'MIME' ,'ATC' ,'ONTO' ,'NSIT' ,'VRNS' ,'STMP' ,'CDKVV' ,'UBNT' ,'SHLS' ,'DV' ,'TIXT' ,'LPSN' ,'MTSI' ,'ACMR' ,'MANT' ,'TLS' ,'NEWR' ,'CCMP' ,'DNJR' ,'SLTD' ,'SLAB' ,'ALRM' ,'CNXC' ,'POWI' ,'CLSK' ,'ALGM' ,'GIB' ,'SUMO' ,'SPT' ,'WK' ,'TSAI' ,'KN' ,'NTCT' ,'NICEY' ,'VRNT' ,'AMKR' ,'JAMF' ,'CALX' ,'SANM' ,'TTEC' ,'MOGO' ,'VSH' ,'COMP' ,'NTGR' ,'ICHR' ,'PING' ,'SVMK' ,'ITRI' ,'PRGS' ,'FORM' ,'TENB' ,'ONTF' ,'ALTR' ,'MAXNV' ,'AVYA' ,'SITM' ,'FN' ,'SNX' ,'MDA' ,'ZUO' ,'EBON' ,'CMBM' ,'TDC' ,'PTAC' ,'SYNA' ,'SWI' ,'CAMT' ,'BLKB' ,'DIO' ,'NATI' ,'SPSC' ,'VZIO' ,'CNDT' ,'SMMC' ,'FTCI' ,'MICT' ,'OLO' ,'EXLS' ,'EGHT' ,'ASX' ,'CSOD' ,'SEMR' ,'WIT' ,'COHU' ,'SGH' ,'PTC' ,'UIS' ,'VCRA' ,'ACVA' ,'NVTL' ,'CGNT' ,'EVTC' ,'VSCI' ,'MLAB' ,'APPF' ,'CMTL' ,'EXTR' ,'EPAY' ,'DSP' ,'PIOS' ,'ATMR' ,'SATS' ,'MCFE' ,'VERI' ,'TSRA' ,'TTMI' ,'ACLS' ,'TSEMF' ,'BNFT' ,'QMCO' ,'MXL' ,'LFUS' ,'RMBS' ,'MODN' ,'NVMI' ,'ROG' ,'INFN' ,'MX' ,'PAMT' ,'FNTE' ,'TBRG' ,'DOCN' ,'BOXL' ,'PLXS' ,'BEEM' ,'INTZ' ,'DLPH' ,'PCVA' ,'NVMM' ,'SMSI' ,'ESE' ,'HSTX' ,'PAYA' ,'LLNW' ,'ONDS' ,'LASR' ,'VERX' ,'AOSL' ,'VICR' ,'GSCN' ,'XGTI' ,'TAIT' ,'MITK' ,'DBD' ,'HLIT' ,'CTS' ,'ATEN' ,'DSGXF' ,'RESN' ,'SMCI' ,'RDWR' ,'SPU' ,'WISA' ,'MSP' ,'CANNY' ,'ALLT' ,'GILTF' ,'BCOV' ,'PI' ,'AKTS' ,'CPSH' ,'INS' ,'CSGS' ,'NPTN' ,'EVOP' ,'SKYT' ,'METHA' ,'MLCH' ,'IMMR' ,'PLAB' ,'BHE' ,'PAAC' ,'AXTI']
June_1 = [BasicMaterials, ConsumerCyclical, FinancialServices, RealEstate, ConsumerDefensive, Healthcare,\
           Utilities, CommunicationServices, Energy, Industrials, Technology]

In [9]:
Stocks_dollar_volume_min_price_continuity = []
for SectorInd in range(len(SectorNames)):
    Stocks_dollar_volume_min_price_continuity.append(intersection([June_1[SectorInd], May_4[SectorInd], April_1[SectorInd]]))
output_str = ""
for SectorInd in range(len(SectorNames)):
    curr_str = SectorNames[SectorInd] + ' = ' + str(Stocks_dollar_volume_min_price_continuity[SectorInd])
    #print (curr_str)
    output_str += curr_str + '\n'
write_file ("Stocks_dollar_volume_min_price_continuity", output_str, option = 'w')

In [10]:
Stocks_dollar_volume_min_price_continuity_all = []
for ticker_lst in Stocks_dollar_volume_min_price_continuity:
    Stocks_dollar_volume_min_price_continuity_all += ticker_lst

In [11]:
print("Number of stocks after price screening (price>20), dollar volume screening (top 70%) and continuity screening \
(April 1st, May 4th and June 1st) is " + str(len(Stocks_dollar_volume_min_price_continuity_all)))

Number of stocks after price screening (price>20), dollar volume screening (top 70%) and continuity screening (April 1st, May 4th and June 1st) is 2547


In [12]:
look_back = '1 M'
resolution = '1 day'
side ='Trades' # last trade, or ASK, or Bid
end_time = datetime.now()
vol_lst = []
for ticker in Stocks_dollar_volume_min_price_continuity_all:
    equity_history = get_stock_data(ticker, look_back, resolution, side, end_time)
    vol_lst.append(calculate_vol(equity_history, ticker))

Error 200, reqId 9: No security definition has been found for the request, contract: Contract(secType='STK', symbol='ABX', exchange='SMART', currency='USD')
Unknown contract: Contract(secType='STK', symbol='ABX', exchange='SMART', currency='USD')


no history available for ticker ABX


Error 200, reqId 20: No security definition has been found for the request, contract: Contract(secType='STK', symbol='PX', exchange='SMART', currency='USD')
Unknown contract: Contract(secType='STK', symbol='PX', exchange='SMART', currency='USD')


no history available for ticker PX


Error 200, reqId 23: No security definition has been found for the request, contract: Contract(secType='STK', symbol='POT', exchange='SMART', currency='USD')
Unknown contract: Contract(secType='STK', symbol='POT', exchange='SMART', currency='USD')


...

no history available for ticker GSCN
no history available for ticker XGTI


Error 200, reqId 4628: No security definition has been found for the request, contract: Contract(secType='STK', symbol='SPU', exchange='SMART', currency='USD')
Unknown contract: Contract(secType='STK', symbol='SPU', exchange='SMART', currency='USD')
Error 200, reqId 4629: No security definition has been found for the request, contract: Contract(secType='STK', symbol='GILTF', exchange='SMART', currency='USD')
Unknown contract: Contract(secType='STK', symbol='GILTF', exchange='SMART', currency='USD')


no history available for ticker SPU
no history available for ticker GILTF


Error 200, reqId 4642: No security definition has been found for the request, contract: Contract(secType='STK', symbol='METHA', exchange='SMART', currency='USD')
Unknown contract: Contract(secType='STK', symbol='METHA', exchange='SMART', currency='USD')
Error 200, reqId 4643: No security definition has been found for the request, contract: Contract(secType='STK', symbol='MLCH', exchange='SMART', currency='USD')
Unknown contract: Contract(secType='STK', symbol='MLCH', exchange='SMART', currency='USD')


no history available for ticker METHA
no history available for ticker MLCH


In [13]:
[Stocks_dollar_volume_min_price_continuity_all_sorted, Vols_dollar_volume_min_price_continuity_all_sorted] = \
sort_first_by_second (Stocks_dollar_volume_min_price_continuity_all, vol_lst)

In [14]:
print("Number of stocks in QuantConnect but can\'t be found in InteractiveBrokers: " + \
      str(len(Stocks_dollar_volume_min_price_continuity_all)-len(Stocks_dollar_volume_min_price_continuity_all_sorted)))
print("Number of stocks after price screening (price>20), dollar volume screening (top 70%), continuity screening \
(April 1st, May 4th and June 1st) and IB availability screening is " + str(len(Stocks_dollar_volume_min_price_continuity_all_sorted)))

Number of stocks in QuantConnect but can't be found in InteractiveBrokers: 449
Number of stocks after price screening (price>20), dollar volume screening (top 70%), continuity screening (April 1st, May 4th and June 1st) and IB availability screening is 2098


In [15]:
output_str = "Stocks_dollar_volume_min_price_continuity_all_sorted = " + str(Stocks_dollar_volume_min_price_continuity_all_sorted)
print (output_str)
write_file ("Stocks_dollar_volume_min_price_continuity_all_sorted", output_str, "w")

Stocks_dollar_volume_min_price_continuity_all_sorted = ['CATM', 'NAV', 'PPD', 'GRA', 'HEC', 'MGLN', 'LMNX', 'RTP', 'RTP', 'FRTA', 'SFTW', 'CAP', 'AACQ', 'PNM', 'PFPT', 'NAD', 'ORBC', 'SPWH', 'SPAQ', 'GPAC', 'VTIQ', 'ATMR', 'NEA', 'STAY', 'TTMI', 'KCAC', 'BCE', 'ERC', 'ALXN', 'VGAC', 'NVS', 'TU', 'KO', 'WPF', 'PRAH', 'FTS', 'PHYS', 'SJR', 'RY', 'FM', 'POST', 'UL', 'IPOF', 'CHNG', 'TDSC', 'JNJ', 'AKAM', 'DNP', 'PFE', 'BPY', 'PEP', 'HSY', 'COHR', 'WLTW', 'TSN', 'EXG', 'BNS', 'UTG', 'GIS', 'CLX', 'BDX', 'CHD', 'TD', 'CDC', 'GSAH', 'CL', 'HRC', 'ADM', 'KHC', 'FTOC', 'VRTX', 'RDY', 'UNH', 'PPL', 'DEO', 'GPN', 'BIP', 'KMB', 'SHW', 'PSA', 'BMO', 'BMY', 'CEF', 'CAG', 'ORLY', 'YUM', 'RBIN', 'MMC', 'ATR', 'K', 'CTSH', 'MCD', 'BUD', 'NI', 'FMC', 'V', 'RGA', 'IDA', 'ABBV', 'CEY', 'PM', 'HAIN', 'ED', 'BTI', 'CPB', 'IBM', 'GOF', 'EL', 'QSR', 'UNP', 'OMC', 'BAX', 'PLXS', 'RDWR', 'DLR', 'CACI', 'WTRG', 'DRE', 'TAK', 'SMFG', 'WORK', 'D', 'FRX', 'MO', 'VRSN', 'MANT', 'DUK', 'RACE', 'SSNC', 'CI', 'HE', 'E

In [16]:
output_str = "Vols_dollar_volume_min_price_continuity_all_sorted = " + str(Vols_dollar_volume_min_price_continuity_all_sorted)
print (output_str)
write_file ("Stocks_dollar_volume_min_price_continuity_all_sorted", "\r\n" + output_str, "a")

Vols_dollar_volume_min_price_continuity_all_sorted = [0.008910088772477235, 0.017437574753168792, 0.017968821361317208, 0.020176651005872452, 0.021671031040704107, 0.026533067484220282, 0.02834618775285369, 0.02906963423305399, 0.030440279794749513, 0.032802633029848864, 0.03750039655301871, 0.038240350501124606, 0.042726972657927784, 0.04665341979020991, 0.049008253711774905, 0.04966386553606998, 0.05083021373651109, 0.053055283323852204, 0.05414255043434378, 0.059096036993610046, 0.06071798223648112, 0.06800913209643325, 0.0704142034533192, 0.07787087673919238, 0.08202310693409415, 0.08231288889918296, 0.08400203498675941, 0.08629257037666611, 0.08832087161334458, 0.08970595035970502, 0.09054628166142117, 0.090817181143399, 0.09237678544046865, 0.09375319334123343, 0.09497903121791566, 0.09504345169456423, 0.09748697316658934, 0.09814653802784595, 0.09839261615902307, 0.098895710829592, 0.10292223929387447, 0.10315640929975821, 0.10445285321528974, 0.10606866755960205, 0.106116519780

In [17]:
lowpercentile = 0.15
highpercentile = 0.85
[Stocks_dollar_volume_min_price_continuity_all_medium_vol, Vols_dollar_volume_min_price_continuity_all_medium_vol] = \
get_medium_vol_stocks (Stocks_dollar_volume_min_price_continuity_all_sorted, Vols_dollar_volume_min_price_continuity_all_sorted\
                      , lowpercentile, highpercentile)

In [18]:
print("Number of stocks after price screening (price>20), dollar volume screening (top 70%), continuity screening \
(April 1st, May 4th and June 1st), IB availability screening and volatility (1 month) screening (15%-85%) is " \
      + str(len(Stocks_dollar_volume_min_price_continuity_all_medium_vol)))

Number of stocks after price screening (price>20), dollar volume screening (top 70%), continuity screening (April 1st, May 4th and June 1st), IB availability screening and volatility (1 month) screening (15%-85%) is 1468


In [19]:
Stocks_dollar_volume_min_price_continuity_medium_vol = \
[intersection([ticker_lst, Stocks_dollar_volume_min_price_continuity_all_medium_vol])\
for ticker_lst in Stocks_dollar_volume_min_price_continuity]

[BasicMaterials, ConsumerCyclical, FinancialServices, RealEstate, ConsumerDefensive, Healthcare,\
 Utilities, CommunicationServices, Energy, Industrials, Technology]\
 = Stocks_dollar_volume_min_price_continuity_medium_vol

output_str = ""
for SectorInd in range(len(SectorNames)):
    curr_str = SectorNames[SectorInd] + ' = ' + str(Stocks_dollar_volume_min_price_continuity_medium_vol[SectorInd])
    print (curr_str)
    output_str += curr_str + '\n'
write_file ("Stocks_dollar_volume_min_price_continuity_medium_vol", output_str, option = 'w')

BasicMaterials = ['RIO', 'FCX', 'X', 'NEM', 'NUE', 'DOW', 'ECL', 'AA', 'DOW', 'AEM', 'MLM', 'LYB', 'CTVA', 'MOS', 'IFF', 'CE', 'AXTA', 'STLD', 'ALB', 'VMC', 'BBL', 'AVTR', 'SMG', 'KGC', 'BHP', 'GGB', 'HUN', 'CF', 'EMN', 'AU', 'KL', 'RGLD', 'UFS', 'AG', 'ASH', 'AUY', 'FNV', 'CX', 'LAC', 'OLN', 'LTHM', 'SUM', 'CXP', 'CC', 'SID', 'BTG', 'WFG', 'RS', 'WLK', 'ESI', 'BCC', 'SWM', 'BVN', 'UFPI', 'SBSW', 'KWR', 'FSM', 'CBT', 'AGI', 'KALU', 'NGVT', 'CMC', 'SAND', 'MDU', 'TROX', 'PKX', 'TSE', 'EGO', 'UNVR', 'WDFC', 'SCHN', 'PVG', 'EQX', 'MTX', 'SILV', 'CSTM', 'KRA', 'GCP', 'IAG', 'CMP', 'OR', 'SVM', 'SUZ', 'SXC', 'SCL', 'NG', 'ASIX']
ConsumerCyclical = ['TSLA', 'AMZN', 'F', 'BABA', 'HD', 'GM', 'DKNG', 'CCL', 'ULTA', 'LOW', 'NCLH', 'JD', 'EBAY', 'BBY', 'GPS', 'TJX', 'MELI', 'LVS', 'W', 'LULU', 'EXPE', 'CMG', 'ETSY', 'RCL', 'CVNA', 'ROST', 'BLL', 'PENN', 'MAR', 'YUMC', 'CROX', 'BURL', 'WYNN', 'IP', 'DHI', 'TSCO', 'VFC', 'PHM', 'WHR', 'RH', 'AAP', 'CHWY', 'POOL', 'DRI', 'LEN', 'HLT', 'TXRH', 'FBHS'